Adding more features in second iteration. Following are the new approaches tried:
<br>
- Imputing the missing values in train and test data.
<br>
- Extracting features from the two transaction files separately
- Mean Encoding the categorical variables instead of One hot encoding.
- Generating additional features based on existing features.

# Google Drive setup 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/My Drive/Applied\ AI\ course\ Assignments/Case\ Study\ 1

/content/drive/My Drive/Applied AI course Assignments/Case Study 1


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

from statsmodels.stats.outliers_influence import variance_inflation_factor

import gc
import warnings
warnings.filterwarnings('ignore')

#plt.style.use('dark_background')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Script to reduce memory usage

In [ ]:
#https://www.kaggle.com/fabiendaniel/elo-world
#Function to load data into pandas and reduce memory usage

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Train and test csv files

In [ ]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

In [ ]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201917 entries, 0 to 201916
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   first_active_month  201917 non-null  object 
 1   card_id             201917 non-null  object 
 2   feature_1           201917 non-null  int64  
 3   feature_2           201917 non-null  int64  
 4   feature_3           201917 non-null  int64  
 5   target              201917 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123623 entries, 0 to 123622
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   first_active_month  123622 non-null  object
 1   card_id             123623 non-null  object
 2   feature_1           123623 non-null  int64 
 3   feature_2           123623 non-null  int64 
 4   feature_3     

So our train data has 201917 cards.
<br> 
We have to make sure we generate features involving these cards only to avoid data leakage.

Referring to Feature Engineering by: https://www.kaggle.com/mfjwr1/simple-lightgbm-without-blending

In [ ]:
train.isna().any()

first_active_month    False
card_id               False
feature_1             False
feature_2             False
feature_3             False
target                False
dtype: bool

In [ ]:
test.isna().any()

first_active_month     True
card_id               False
feature_1             False
feature_2             False
feature_3             False
dtype: bool

In [ ]:
test[test['first_active_month'].isna()]

,first_active_month,card_id,feature_1,feature_2,feature_3
11578,NaN,C_ID_c27b4f80f7,5,2,1


In [ ]:
test['first_active_month'].fillna(test['first_active_month'].mode()[0], inplace=True)

In [ ]:
test.isna().any()

first_active_month    False
card_id               False
feature_1             False
feature_2             False
feature_3             False
dtype: bool

#### Features using first_active_month

In [ ]:
train['first_active_month'] = pd.to_datetime(train['first_active_month'])
train['quarter_first_active_month'] = train['first_active_month'].dt.quarter
train['first_active_month_diff_from_today'] = (datetime.datetime.today() - train['first_active_month']).dt.days

test['first_active_month'] = pd.to_datetime(test['first_active_month'])
test['quarter_first_active_month'] = test['first_active_month'].dt.quarter
test['first_active_month_diff_from_today'] = (datetime.datetime.today() - test['first_active_month']).dt.days

In [ ]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,quarter_first_active_month,first_active_month_diff_from_today
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,2,1215
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,1,1366
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,3,1519
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,3,1123
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,4,1062


In [ ]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,quarter_first_active_month,first_active_month_diff_from_today
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,2,1276
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,1,1366
2,2017-08-01,C_ID_b709037bc5,5,1,1,3,1154
3,2017-12-01,C_ID_d27d835a9f,2,1,0,4,1032
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1,4,1763


### Labeling rare data points 

Using mean encoding for categorical features points to predict rare data points better

In [ ]:
train['rare_datapoints'] = 0
train.loc[train['target'] < -30, 'rare_datapoints'] = 1

In [ ]:
train['rare_datapoints'].value_counts()

0    199710
1      2207
Name: rare_datapoints, dtype: int64

In [ ]:
for f in ['feature_1', 'feature_2', 'feature_3']:
  rare_data_mean = train.groupby([f])['rare_datapoints'].mean()
  train[f] = train[f].map(rare_data_mean)
  test[f] = test[f].map(rare_data_mean)

In [ ]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,quarter_first_active_month,first_active_month_diff_from_today,rare_datapoints
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.820283,2,1215,0
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.392913,1,1366,0
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.688056,3,1519,0
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,0.142495,3,1123,0
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,-0.159749,4,1062,0


In [ ]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,quarter_first_active_month,first_active_month_diff_from_today
0,2017-04-01,C_ID_0ab67a22ab,0.010479,0.014166,0.011428,2,1276
1,2017-01-01,C_ID_130fd0cbdd,0.010610,0.014166,0.010283,1,1366
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011385,0.011428,3,1154
3,2017-12-01,C_ID_d27d835a9f,0.010610,0.011385,0.010283,4,1032
4,2015-12-01,C_ID_2b5e3df5c2,0.013145,0.011385,0.011428,4,1763


#### Combining categorical features and date features

In [ ]:
test['target'] = np.nan
#Products
train['cat_time_feature1'] = train['first_active_month_diff_from_today'] * train['feature_1']
train['cat_time_feature2'] = train['first_active_month_diff_from_today'] * train['feature_2']
train['cat_time_feature3'] = train['first_active_month_diff_from_today'] * train['feature_3']

test['cat_time_feature1'] = test['first_active_month_diff_from_today'] * test['feature_1']
test['cat_time_feature2'] = test['first_active_month_diff_from_today'] * test['feature_2']
test['cat_time_feature3'] = test['first_active_month_diff_from_today'] * test['feature_3']

#Ratios
train['cat_time_ratio1'] = train['feature_1'] / train['first_active_month_diff_from_today']
train['cat_time_ratio2'] = train['feature_2'] / train['first_active_month_diff_from_today']
train['cat_time_ratio3'] = train['feature_3'] / train['first_active_month_diff_from_today']

test['cat_time_ratio1'] = test['feature_1'] / test['first_active_month_diff_from_today']
test['cat_time_ratio2'] = test['feature_2'] / test['first_active_month_diff_from_today']
test['cat_time_ratio3'] = test['feature_3'] / test['first_active_month_diff_from_today']

In [ ]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,quarter_first_active_month,first_active_month_diff_from_today,rare_datapoints,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.820283,2,1215,0,15.970708,10.633827,13.884438,0.000011,0.000007,0.000009
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.392913,1,1366,0,14.632034,15.551601,14.046353,0.000008,0.000008,0.000008
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.688056,3,1519,0,16.116422,13.294472,15.619626,0.000007,0.000006,0.000007
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,0.142495,3,1123,0,12.029117,15.908870,11.547624,0.000010,0.000013,0.000009
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,-0.159749,4,1062,0,8.558112,15.044719,10.920371,0.000008,0.000013,0.000010


### Aggregation based features on categorical features

In [ ]:
train['feature_sum'] = train['feature_1'] + train['feature_2'] + train['feature_3']
train['feature_mean'] = train['feature_sum']/3
train['feature_max'] = train[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
train['feature_min'] = train[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
train['feature_std'] = train[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

feature_cols = ['feature_1', 'feature_2', 'feature_3']
for f in feature_cols:
    train['days_' + f] = train['first_active_month_diff_from_today'] * train[f]
    train['days_' + f + '_ratio'] = train[f] / train['first_active_month_diff_from_today']

test['feature_sum'] = test['feature_1'] + test['feature_2'] + test['feature_3']
test['feature_mean'] = test['feature_sum']/3
test['feature_max'] = test[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
test['feature_min'] = test[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
test['feature_std'] = test[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

feature_cols = ['feature_1', 'feature_2', 'feature_3']
for f in feature_cols:
    test['days_' + f] = test['first_active_month_diff_from_today'] * test[f]
    test['days_' + f + '_ratio'] = test[f] / test['first_active_month_diff_from_today']

In [ ]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,quarter_first_active_month,first_active_month_diff_from_today,rare_datapoints,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio
0,2017-06-01,C_ID_92a2005557,0.013145,0.008752,0.011428,-0.820283,2,1215,0,15.970708,10.633827,13.884438,0.000011,0.000007,0.000009,0.033324,0.011108,0.013145,0.008752,0.002214,15.970708,0.000011,10.633827,0.000007,13.884438,0.000009
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011385,0.010283,0.392913,1,1366,0,14.632034,15.551601,14.046353,0.000008,0.000008,0.000008,0.032379,0.010793,0.011385,0.010283,0.000555,14.632034,0.000008,15.551601,0.000008,14.046353,0.000008
2,2016-08-01,C_ID_d639edf6cd,0.010610,0.008752,0.010283,0.688056,3,1519,0,16.116422,13.294472,15.619626,0.000007,0.000006,0.000007,0.029645,0.009882,0.010610,0.008752,0.000992,16.116422,0.000007,13.294472,0.000006,15.619626,0.000007
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014166,0.010283,0.142495,3,1123,0,12.029117,15.908870,11.547624,0.000010,0.000013,0.000009,0.035161,0.011720,0.014166,0.010283,0.002129,12.029117,0.000010,15.908870,0.000013,11.547624,0.000009
4,2017-11-01,C_ID_cdbd2c0db2,0.008058,0.014166,0.010283,-0.159749,4,1062,0,8.558112,15.044719,10.920371,0.000008,0.000013,0.000010,0.032508,0.010836,0.014166,0.008058,0.003091,8.558112,0.000008,15.044719,0.000013,10.920371,0.000010


In [ ]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,quarter_first_active_month,first_active_month_diff_from_today,target,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio
0,2017-04-01,C_ID_0ab67a22ab,0.010479,0.014166,0.011428,2,1276,NaN,13.371699,18.076329,14.581516,0.000008,0.000011,0.000009,0.036073,0.012024,0.014166,0.010479,0.001915,13.371699,0.000008,18.076329,0.000011,14.581516,0.000009
1,2017-01-01,C_ID_130fd0cbdd,0.010610,0.014166,0.010283,1,1366,NaN,14.493109,19.351306,14.046353,0.000008,0.000010,0.000008,0.035059,0.011686,0.014166,0.010283,0.002154,14.493109,0.000008,19.351306,0.000010,14.046353,0.000008
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011385,0.011428,3,1154,NaN,15.168886,13.138029,13.187359,0.000011,0.000010,0.000010,0.035957,0.011986,0.013145,0.011385,0.001004,15.168886,0.000011,13.138029,0.000010,13.187359,0.000010
3,2017-12-01,C_ID_d27d835a9f,0.010610,0.011385,0.010283,4,1032,NaN,10.949406,11.749087,10.611886,0.000010,0.000011,0.000010,0.032277,0.010759,0.011385,0.010283,0.000566,10.949406,0.000010,11.749087,0.000011,10.611886,0.000010
4,2015-12-01,C_ID_2b5e3df5c2,0.013145,0.011385,0.011428,4,1763,NaN,23.173957,20.071357,20.146719,0.000007,0.000006,0.000006,0.035957,0.011986,0.013145,0.011385,0.001004,23.173957,0.000007,20.071357,0.000006,20.146719,0.000006


In [ ]:
train.isna().any().sum()

0

In [ ]:
test.isna().any().sum() #Checking except for target which is nan

1

In [ ]:
train.to_csv('New_Data_processed/processed_train.csv')
test.to_csv('New_Data_processed/processed_test.csv')

# Transaction Files

## 1. historical_transactions 

In [ ]:
historical_transactions = reduce_mem_usage(pd.read_csv('Data/historical_transactions.csv'))
historical_transactions.replace([-np.inf, np.inf], np.nan, inplace=True)

Mem. usage decreased to 1749.11 Mb (43.7% reduction)


In [ ]:
historical_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29112361 entries, 0 to 29112360
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   authorized_flag       object 
 1   card_id               object 
 2   city_id               int16  
 3   category_1            object 
 4   installments          int16  
 5   category_3            object 
 6   merchant_category_id  int16  
 7   merchant_id           object 
 8   month_lag             int8   
 9   purchase_amount       float32
 10  purchase_date         object 
 11  category_2            float16
 12  state_id              int8   
 13  subsector_id          int8   
dtypes: float16(1), float32(1), int16(3), int8(3), object(6)
memory usage: 1.7+ GB


In [ ]:
#all_transactions['installments'].replace(-1, np.nan, inplace=True)
#all_transactions['installments'].replace(999, np.nan, inplace=True)
#all_transactions['installments'].fillna(all_transactions['installments'].mode()[0], inplace=True)

In [ ]:
historical_transactions.isna().any()

authorized_flag         False
card_id                 False
city_id                 False
category_1              False
installments            False
category_3               True
merchant_category_id    False
merchant_id              True
month_lag               False
purchase_amount         False
purchase_date           False
category_2               True
state_id                False
subsector_id            False
dtype: bool

Imputing the missing values

In [ ]:
historical_transactions['merchant_id'].fillna(historical_transactions['merchant_id'].mode()[0], inplace=True)
historical_transactions['category_3'].fillna(historical_transactions['category_3'].mode()[0], inplace=True)
historical_transactions['category_2'].fillna(historical_transactions['category_2'].mode()[0], inplace=True)

In [ ]:
historical_transactions['purchase_amount'] = historical_transactions['purchase_amount'].apply(lambda x: min(x, 0.8))
historical_transactions['installments'].replace([-1, 999], np.nan, inplace=True)
historical_transactions['installments'].fillna(historical_transactions['installments'].mode()[0], inplace=True)
historical_transactions['price'] = historical_transactions['purchase_amount'] / (historical_transactions['installments'] + 0.01)

In [ ]:
historical_transactions.isna().any()

authorized_flag         False
card_id                 False
city_id                 False
category_1              False
installments            False
category_3              False
merchant_category_id    False
merchant_id             False
month_lag               False
purchase_amount         False
purchase_date           False
category_2              False
state_id                False
subsector_id            False
price                   False
dtype: bool

In [ ]:
historical_transactions['authorized_flag'] = historical_transactions['authorized_flag'].map({'Y': 1, 'N': 0})
historical_transactions['category_1'] = historical_transactions['category_1'].map({'Y': 1, 'N': 0})

In [ ]:
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,price
0,1,C_ID_4e6213e9bc,88,0,0.0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,-70.333093
1,1,C_ID_4e6213e9bc,88,0,0.0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,-73.312849
2,1,C_ID_4e6213e9bc,88,0,0.0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,-72.038603
3,1,C_ID_4e6213e9bc,88,0,0.0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,-73.535240
4,1,C_ID_4e6213e9bc,88,0,0.0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,-72.286540


In [ ]:
historical_transactions.to_csv('New_Data_processed/historical_transactions_processed.csv')

### Featurization using purchase date

Referring the EDA kernel on dates: https://www.kaggle.com/denzo123/a-closer-look-at-date-variables

In [ ]:
historical_transactions = reduce_mem_usage(pd.read_csv('New_Data_processed/historical_transactions_processed.csv', index_col=0))

Mem. usage decreased to 1582.53 Mb (52.5% reduction)


In [ ]:
historical_transactions = historical_transactions[['card_id', 'purchase_date', 'month_lag']]

In [ ]:
pd.to_datetime(historical_transactions['purchase_date'], format='%Y-%m-%d %H:%M:%S').dt.year.value_counts()

2017    23828069
2018     5284292
Name: purchase_date, dtype: int64

So our data contains transactions from two years: 2017 and 2018.

In [ ]:
historical_transactions['purchase_date'] = pd.to_datetime(historical_transactions['purchase_date'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
# Get the day of the week for the purchase date
def get_weekday(data):
  return data.dt.dayofweek

# Return 1 if the purchase date is on a weekend
def is_weekend(day):
  if day == 5 or day == 6:
    return 1
  else:
    return 0

#Return day of purchase
def get_day(date_obj):
  return date_obj.dt.day

#Return week of the year of purchase
def get_week_of_year(date_obj):
  return date_obj.dt.weekofyear

#Return hour of purchase
def get_hour(date_obj):
  return date_obj.dt.hour

#Return month of purchase date
def get_purchase_month(data):
  return data.dt.month

#Return the month phase during the purchase
#Eg. Early in the month, mid-month or at the end of the month
def get_time_of_month(date):
  if date.day <=10:
    return "Early"
  elif date.day > 10 and date.day <= 20:
    return "Mid"
  else:
    return "End"
  

# Time of the day during purchase
#Eg. Morning, Afternoon, Evening, Night
def get_time_of_day(time):
  if time.hour >= 4 and time.hour < 12:
    return "Morning"
  elif time.hour >= 12 and time.hour < 17:
    return "Afternoon"
  elif time.hour >= 17 and time.hour < 22:
    return "Evening"
  else:
    return "Night"
    

# Returns 1 if the purchase was made on a holiday(Saturdays and sundays excluded)
# Google Search : list of holidays in brazil 2017 and 2018
def get_isholiday(date):
  holiday_list=[
            '01-01-17', '14-02-17', '28-08-17', '14-04-17', '16-04-17', '21-04-17',
            '01-05-17', '15-06-17', '07-09-17', '12-10-17', '02-11-17', '15-11-17', 
            '24-12-17', '25-12-17', '31-12-17',
            '01-01-18', '14-02-18', '28-08-18', '14-04-18', '16-04-18', '21-04-18',
            '01-05-18', '15-06-18', '07-09-18', '12-10-18', '02-11-18', '15-11-18', 
            '24-12-18', '25-12-18', '31-12-18'
  ]
  date = date.strftime(format='%d-%m-%y') 
  if date in holiday_list:
    return 1
  else:
    return 0

In [ ]:
historical_transactions.head()

,card_id,purchase_date,month_lag
0,C_ID_4e6213e9bc,2017-06-25 15:33:07,-8
1,C_ID_4e6213e9bc,2017-07-15 12:10:45,-7
2,C_ID_4e6213e9bc,2017-08-09 22:04:29,-6
3,C_ID_4e6213e9bc,2017-09-02 10:06:26,-5
4,C_ID_4e6213e9bc,2017-03-10 01:14:19,-11


In [ ]:
historical_transactions['weekday'] = get_weekday(historical_transactions['purchase_date'])
historical_transactions['is_weekend'] = historical_transactions['weekday'].apply(lambda day: is_weekend(day))
historical_transactions['purchase_month'] = get_purchase_month(historical_transactions['purchase_date'])
historical_transactions['purchase_day'] = get_day(historical_transactions['purchase_date'])
historical_transactions['week_of_year'] = get_week_of_year(historical_transactions['purchase_date'])
historical_transactions['purchase_hour'] = get_hour(historical_transactions['purchase_date'])
#historical_transactions['purchase_time_of_day'] = historical_transactions['purchase_date'].apply(lambda day: get_time_of_day(day))
#historical_transactions['purchase_month_time'] = historical_transactions['purchase_date'].apply(lambda date: get_time_of_month(date))
historical_transactions['purchase_on_holiday'] = historical_transactions['purchase_date'].apply(lambda date_obj: get_isholiday(date_obj))
historical_transactions['purchase_date'] = historical_transactions['purchase_date'].dt.date

In [ ]:
historical_transactions['month_diff'] = ((datetime.date.today() - historical_transactions['purchase_date']).dt.days)//30
historical_transactions['month_diff'] +=historical_transactions['month_lag']
del historical_transactions['month_lag']

Saving the dataframe due to memory constraints.

In [ ]:
historical_transactions.to_csv('New_Data_processed/historical_transactions_date_features.csv')

#### Aggregating:

In [ ]:
historical_transactions = pd.read_csv('New_Data_processed/historical_transactions_date_features.csv', index_col=0)

In [ ]:
historical_transactions.head()

In [ ]:
historical_transactions['purchase_date'] = pd.to_datetime(historical_transactions['purchase_date'])

In [ ]:
aggregations = {
    'is_weekend': ['sum', 'mean'],
    'purchase_on_holiday': ['sum', 'mean'],
    'weekday' : ['nunique', 'sum', 'mean'],
    'purchase_hour': ['nunique', 'mean', 'min', 'max'],
    'week_of_year': ['nunique', 'mean', 'min', 'max'],
    'month_diff': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'purchase_day': ['nunique', 'sum', 'min'],
    'purchase_date' : [np.ptp, 'min', 'max'],
    'purchase_month' : ['sum', 'mean', 'nunique']

}

aggregated_date_features = historical_transactions.groupby('card_id').agg(aggregations)
aggregated_date_features.columns = ['transactions_'+'_'.join(col).strip() 
                           for col in aggregated_date_features.columns.values]

In [ ]:
aggregated_date_features.head()

,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,transactions_week_of_year_max,transactions_month_diff_sum,transactions_month_diff_mean,transactions_month_diff_min,transactions_month_diff_max,transactions_month_diff_var,transactions_month_diff_skew,transactions_purchase_day_nunique,transactions_purchase_day_sum,transactions_purchase_day_min,transactions_purchase_date_ptp,transactions_purchase_date_min,transactions_purchase_date_max,transactions_purchase_month_sum,transactions_purchase_month_mean,transactions_purchase_month_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,25,0.167785,7,0.046980,7,334,2.241611,18,14.416107,0,22,39,25.550336,1,52,4718,31.664430,31,32,0.224469,-0.703558,28,1974,1,378 days,2017-02-14,2018-02-27,950,6.375839,12
C_ID_0001238066,52,0.422764,8,0.065041,7,431,3.504065,20,14.739837,0,23,23,29.967480,1,52,3857,31.357724,31,32,0.231641,0.601001,30,1985,1,152 days,2017-09-28,2018-02-27,899,7.308943,6
C_ID_0001506ef0,32,0.484848,0,0.000000,7,236,3.575758,15,12.606061,0,21,24,27.090909,1,51,2093,31.712121,31,32,0.208159,-0.958917,25,792,1,399 days,2017-01-14,2018-02-17,449,6.803030,11
C_ID_0001793786,37,0.171296,10,0.046296,7,582,2.694444,21,15.606481,0,23,33,27.134259,3,44,7676,35.537037,35,36,0.249785,-0.149597,31,3481,1,283 days,2017-01-21,2017-10-31,1441,6.671296,10
C_ID_000183fdda,33,0.229167,1,0.006944,7,423,2.937500,19,16.465278,0,23,27,27.881944,1,52,4544,31.555556,31,32,0.248640,-0.225967,30,2021,1,202 days,2017-08-07,2018-02-25,991,6.881944,7


In [ ]:
aggregated_date_features['transactions_purchase_date_ptp'] = aggregated_date_features['transactions_purchase_date_ptp'].dt.days

In [ ]:
d_now = datetime.datetime.today()
d_now

datetime.datetime(2020, 9, 29, 8, 19, 22, 855768)

In [ ]:
aggregated_date_features['transactions_purchase_date_max_diff_now'] = (datetime.datetime.today() - aggregated_date_features['transactions_purchase_date_max']).dt.days

In [ ]:
aggregated_date_features['transactions_purchase_date_min_diff_now'] = (datetime.datetime.today() - aggregated_date_features['transactions_purchase_date_min']).dt.days

In [ ]:
del aggregated_date_features['transactions_purchase_date_ptp']

In [ ]:
aggregated_date_features.head()

,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,transactions_week_of_year_max,transactions_month_diff_sum,transactions_month_diff_mean,transactions_month_diff_min,transactions_month_diff_max,transactions_month_diff_var,transactions_month_diff_skew,transactions_purchase_day_nunique,transactions_purchase_day_sum,transactions_purchase_day_min,transactions_purchase_date_min,transactions_purchase_date_max,transactions_purchase_month_sum,transactions_purchase_month_mean,transactions_purchase_month_nunique,transactions_purchase_date_max_diff_now,transactions_purchase_date_min_diff_now
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,25,0.167785,7,0.046980,7,334,2.241611,18,14.416107,0,22,39,25.550336,1,52,4718,31.664430,31,32,0.224469,-0.703558,28,1974,1,2017-02-14,2018-02-27,950,6.375839,12,945,1323
C_ID_0001238066,52,0.422764,8,0.065041,7,431,3.504065,20,14.739837,0,23,23,29.967480,1,52,3857,31.357724,31,32,0.231641,0.601001,30,1985,1,2017-09-28,2018-02-27,899,7.308943,6,945,1097
C_ID_0001506ef0,32,0.484848,0,0.000000,7,236,3.575758,15,12.606061,0,21,24,27.090909,1,51,2093,31.712121,31,32,0.208159,-0.958917,25,792,1,2017-01-14,2018-02-17,449,6.803030,11,955,1354
C_ID_0001793786,37,0.171296,10,0.046296,7,582,2.694444,21,15.606481,0,23,33,27.134259,3,44,7676,35.537037,35,36,0.249785,-0.149597,31,3481,1,2017-01-21,2017-10-31,1441,6.671296,10,1064,1347
C_ID_000183fdda,33,0.229167,1,0.006944,7,423,2.937500,19,16.465278,0,23,27,27.881944,1,52,4544,31.555556,31,32,0.248640,-0.225967,30,2021,1,2017-08-07,2018-02-25,991,6.881944,7,947,1149


In [ ]:
aggregated_date_features.to_csv('New_Data_processed/historical_transactions_date_features_final.csv')

### Other Features from transactions

In [ ]:
historical_transactions = reduce_mem_usage(pd.read_csv('New_Data_processed/historical_transactions_processed.csv', index_col = 0))

Mem. usage decreased to 1582.53 Mb (55.5% reduction)


In [ ]:
date_features = reduce_mem_usage(pd.read_csv('New_Data_processed/historical_transactions_date_features.csv', index_col = 0))

Mem. usage decreased to 888.44 Mb (63.6% reduction)


In [ ]:
month_diff = date_features['month_diff']

In [ ]:
historical_transactions['duration'] = historical_transactions['purchase_amount']*month_diff
historical_transactions['amount_month_ratio'] = historical_transactions['purchase_amount']/month_diff

In [ ]:
del month_diff,date_features

gc.collect()

0

In [ ]:
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,price,duration,amount_month_ratio
0,1,C_ID_4e6213e9bc,88,0,0.0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,-70.333093,-21.803259,-0.022688
1,1,C_ID_4e6213e9bc,88,0,0.0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,-73.312849,-23.460112,-0.022910
2,1,C_ID_4e6213e9bc,88,0,0.0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,-72.038603,-23.052353,-0.022512
3,1,C_ID_4e6213e9bc,88,0,0.0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,-73.535240,-23.531277,-0.022980
4,1,C_ID_4e6213e9bc,88,0,0.0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,-72.286540,-23.131693,-0.022590


In [ ]:
cat_aggregations = pd.DataFrame()
for col in ['category_2', 'category_3']:
  cat_aggregations[col + '_mean'] = historical_transactions.groupby([col])['purchase_amount'].transform('mean')
  cat_aggregations[col + '_min'] = historical_transactions.groupby([col])['purchase_amount'].transform('min')
  cat_aggregations[col + '_max'] = historical_transactions.groupby([col])['purchase_amount'].transform('max')
  cat_aggregations[col + '_sum'] = historical_transactions.groupby([col])['purchase_amount'].transform('sum')

In [ ]:
cat_aggregations['card_id'] = historical_transactions['card_id']

In [ ]:
cat_aggregations.head()

,category_2_mean,category_2_min,category_2_max,category_2_sum,category_3_mean,category_3_min,category_3_max,category_3_sum,card_id
0,-0.606656,-0.746908,0.8,-1.081671e+07,-0.639589,-0.746908,0.8,-9.971130e+06,C_ID_4e6213e9bc
1,-0.606656,-0.746908,0.8,-1.081671e+07,-0.639589,-0.746908,0.8,-9.971130e+06,C_ID_4e6213e9bc
2,-0.606656,-0.746908,0.8,-1.081671e+07,-0.639589,-0.746908,0.8,-9.971130e+06,C_ID_4e6213e9bc
3,-0.606656,-0.746908,0.8,-1.081671e+07,-0.639589,-0.746908,0.8,-9.971130e+06,C_ID_4e6213e9bc
4,-0.606656,-0.746908,0.8,-1.081671e+07,-0.639589,-0.746908,0.8,-9.971130e+06,C_ID_4e6213e9bc


In [ ]:
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,price,duration,amount_month_ratio
0,1,C_ID_4e6213e9bc,88,0,0.0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,-70.333093,-21.803259,-0.022688
1,1,C_ID_4e6213e9bc,88,0,0.0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,-73.312849,-23.460112,-0.022910
2,1,C_ID_4e6213e9bc,88,0,0.0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,-72.038603,-23.052353,-0.022512
3,1,C_ID_4e6213e9bc,88,0,0.0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,-73.535240,-23.531277,-0.022980
4,1,C_ID_4e6213e9bc,88,0,0.0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,-72.286540,-23.131693,-0.022590


In [ ]:
cat_aggregations.to_csv('New_Data_processed/transactions_category_aggregations_features.csv')

In [ ]:
del cat_aggregations
gc.collect()

24

In [ ]:
historical_transactions.drop(columns=['purchase_date'], axis=1, inplace=True)

In [ ]:
historical_transactions.to_csv('New_Data_processed/historical_transactions_processed.csv')

Saving the dataframe due to memory constraints

#### Aggregating the category features generated using transform

In [ ]:
cat_aggregations = reduce_mem_usage(pd.read_csv('New_Data_processed/transactions_category_aggregations_features.csv'))

Mem. usage decreased to 888.44 Mb (60.0% reduction)


In [ ]:
aggregated = cat_aggregations.groupby('card_id').mean()

In [ ]:
aggregated.head()

,Unnamed: 0,category_2_mean,category_2_min,category_2_max,category_2_sum,category_3_mean,category_3_min,category_3_max,category_3_sum
card_id,,,,,,,,,
C_ID_00007093c1,19095848.0,-0.593750,-0.74707,0.799805,-3908397.50,-0.552246,-0.74707,0.799805,-6147802.5
C_ID_0001238066,26539170.0,-0.605957,-0.74707,0.799805,-9423409.00,-0.509766,-0.74707,0.799805,-5528821.5
C_ID_0001506ef0,25106432.5,-0.591309,-0.74707,0.799805,-2568065.25,-0.639160,-0.74707,0.799805,-9930074.0
C_ID_0001793786,12034801.5,-0.601562,-0.74707,0.799805,-6635398.00,-0.639160,-0.74707,0.799805,-9908405.0
C_ID_000183fdda,6867570.5,-0.592285,-0.74707,0.799805,-2947862.75,-0.511230,-0.74707,0.799805,-5560250.0


In [ ]:
aggregated.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
aggregated.to_csv('New_Data_processed/transactions_category_aggregated_final.csv')

#### Numerical Features aggregation

In [ ]:
historical_transactions = reduce_mem_usage(pd.read_csv('New_Data_processed/historical_transactions_processed.csv', index_col=0))

Mem. usage decreased to 1471.48 Mb (61.0% reduction)


In [ ]:
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,category_2,state_id,subsector_id,price,duration,amount_month_ratio
0,1,C_ID_4e6213e9bc,88,0,0.0,A,80,M_ID_e020e9b302,-8,-0.703125,1.0,16,37,-70.3125,-21.796875,-0.022690
1,1,C_ID_4e6213e9bc,88,0,0.0,A,367,M_ID_86ec983688,-7,-0.732910,1.0,16,16,-73.3125,-23.453125,-0.022903
2,1,C_ID_4e6213e9bc,88,0,0.0,A,80,M_ID_979ed661fc,-6,-0.720215,1.0,16,37,-72.0625,-23.046875,-0.022507
3,1,C_ID_4e6213e9bc,88,0,0.0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,1.0,16,34,-73.5625,-23.531250,-0.022980
4,1,C_ID_4e6213e9bc,88,0,0.0,A,80,M_ID_e020e9b302,-11,-0.722656,1.0,16,37,-72.3125,-23.125000,-0.022583


#### Aggregating the columns based on card_id

In [ ]:
aggregations = {
    'authorized_flag' : ['sum', 'mean'],
    'category_1' : ['sum', 'mean'],
    'card_id': ['size'],
    'city_id' : ['nunique'],
    'state_id' : ['nunique'],
    'subsector_id' : ['nunique'],
    'merchant_category_id' : ['nunique'],
    'merchant_id': ['nunique'],
    'month_lag' : ['sum', 'mean', 'min', 'max', 'var'],
    'duration': ['mean', 'min', 'mean', 'max', 'var', 'skew'],
    'amount_month_ratio': ['mean', 'min', 'max', 'var', 'skew'],
    'installments' : ['sum', 'mean', 'min', 'max', 'var'],
    'purchase_amount' : ['sum', 'mean', 'min', 'max', 'var'],
    'price': ['sum', 'mean', 'min', 'max', 'var', 'skew']
    
}

aggregated_numerical_features = historical_transactions.groupby('card_id').agg(aggregations)
aggregated_numerical_features.columns = ['transactions_'+'_'.join(col).strip() 
                           for col in aggregated_numerical_features.columns.values]

In [ ]:
aggregated_numerical_features.head()

,transactions_authorized_flag_sum,transactions_authorized_flag_mean,transactions_category_1_sum,transactions_category_1_mean,transactions_card_id_size,transactions_city_id_nunique,transactions_state_id_nunique,transactions_subsector_id_nunique,transactions_merchant_category_id_nunique,transactions_merchant_id_nunique,transactions_month_lag_sum,transactions_month_lag_mean,transactions_month_lag_min,transactions_month_lag_max,transactions_month_lag_var,transactions_duration_mean,transactions_duration_min,transactions_duration_mean,transactions_duration_max,transactions_duration_var,transactions_duration_skew,transactions_amount_month_ratio_mean,transactions_amount_month_ratio_min,transactions_amount_month_ratio_max,transactions_amount_month_ratio_var,transactions_amount_month_ratio_skew,transactions_installments_sum,transactions_installments_mean,transactions_installments_min,transactions_installments_max,transactions_installments_var,transactions_purchase_amount_sum,transactions_purchase_amount_mean,transactions_purchase_amount_min,transactions_purchase_amount_max,transactions_purchase_amount_var,transactions_price_sum,transactions_price_mean,transactions_price_min,transactions_price_max,transactions_price_var,transactions_price_skew
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,114.0,0.765101,28.0,0.187919,149,4,3,13,18,29,-872.0,-5.852349,-12,0,11.923998,-16.593750,-23.328125,-16.593750,24.796875,64.31250,inf,-0.016495,-0.023270,0.025803,0.000065,2.851562,192.0,1.289062,1.0,6.0,0.584961,-77.93750,-0.523438,-0.729004,0.799805,0.064514,-73.875,-0.495850,-0.72168,0.304932,0.058563,1.458984
C_ID_0001238066,120.0,0.975610,2.0,0.016260,123,18,6,17,29,65,-223.0,-1.813008,-5,0,1.661469,-18.484375,-23.421875,-18.484375,23.812500,35.40625,inf,-0.018784,-0.023712,0.024780,0.000037,4.054688,201.0,1.633789,0.0,10.0,2.103516,-72.43750,-0.588867,-0.734863,0.768066,0.036194,-114.125,-0.927734,-60.50000,24.484375,36.500000,-inf
C_ID_0001506ef0,62.0,0.939394,0.0,0.000000,66,3,2,12,19,28,-319.0,-4.833333,-13,0,17.956410,-16.968750,-23.703125,-16.968750,25.593750,185.62500,inf,-0.016953,-0.023849,0.024994,0.000180,2.402344,1.0,0.015152,0.0,1.0,0.015152,-35.40625,-0.536621,-0.740723,0.799805,0.182739,-3470.000,-52.593750,-74.06250,80.000000,1865.000000,inf
C_ID_0001793786,189.0,0.875000,2.0,0.009259,216,10,4,24,48,119,-719.0,-3.328704,-9,0,5.319358,-10.039062,-26.093750,-10.039062,28.796875,295.50000,1.294922,-0.007942,-0.021301,0.022858,0.000185,1.293945,5.0,0.023148,0.0,1.0,0.022720,-60.96875,-0.282227,-0.745605,0.799805,0.234009,-5760.000,-26.671875,-73.81250,80.000000,2316.000000,NaN
C_ID_000183fdda,137.0,0.951389,4.0,0.027778,144,9,7,21,36,73,-353.0,-2.451389,-6,0,3.592026,-16.203125,-23.609375,-16.203125,25.593750,134.50000,inf,-0.016281,-0.023758,0.025803,0.000134,2.541016,268.0,1.861328,0.0,10.0,4.359375,-73.93750,-0.513672,-0.737793,0.799805,0.134399,-228.750,-1.588867,-53.65625,0.791992,51.312500,-inf


In [ ]:
aggregated_numerical_features.to_csv('New_Data_processed/transactions_other_features_final.csv')

## 2. new_merchant_transactions

In [ ]:
new_merchant_transactions = reduce_mem_usage(pd.read_csv('Data/new_merchant_transactions.csv'))
new_merchant_transactions.replace([-np.inf, np.inf], np.nan, inplace=True)

Mem. usage decreased to 114.20 Mb (45.5% reduction)


In [ ]:
new_merchant_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1963031 entries, 0 to 1963030
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   authorized_flag       object 
 1   card_id               object 
 2   city_id               int16  
 3   category_1            object 
 4   installments          int16  
 5   category_3            object 
 6   merchant_category_id  int16  
 7   merchant_id           object 
 8   month_lag             int8   
 9   purchase_amount       float16
 10  purchase_date         object 
 11  category_2            float16
 12  state_id              int8   
 13  subsector_id          int8   
dtypes: float16(2), int16(3), int8(3), object(6)
memory usage: 114.2+ MB


In [ ]:
new_merchant_transactions.isna().any()

authorized_flag         False
card_id                 False
city_id                 False
category_1              False
installments            False
category_3               True
merchant_category_id    False
merchant_id              True
month_lag               False
purchase_amount         False
purchase_date           False
category_2               True
state_id                False
subsector_id            False
dtype: bool

Imputing the missing values

In [ ]:
new_merchant_transactions['merchant_id'].fillna(new_merchant_transactions['merchant_id'].mode()[0], inplace=True)
new_merchant_transactions['category_3'].fillna(new_merchant_transactions['category_3'].mode()[0], inplace=True)
new_merchant_transactions['category_2'].fillna(new_merchant_transactions['category_2'].mode()[0], inplace=True)

In [ ]:
new_merchant_transactions['purchase_amount'] = new_merchant_transactions['purchase_amount'].apply(lambda x: min(x, 0.8))
new_merchant_transactions['installments'].replace([-1, 999], np.nan, inplace=True)
new_merchant_transactions['installments'].fillna(new_merchant_transactions['installments'].mode()[0], inplace=True)
new_merchant_transactions['price'] = new_merchant_transactions['purchase_amount'] / (new_merchant_transactions['installments'] + 0.01)

In [ ]:
new_merchant_transactions.isna().any()

authorized_flag         False
card_id                 False
city_id                 False
category_1              False
installments            False
category_3              False
merchant_category_id    False
merchant_id             False
month_lag               False
purchase_amount         False
purchase_date           False
category_2              False
state_id                False
subsector_id            False
price                   False
dtype: bool

In [ ]:
new_merchant_transactions['authorized_flag'] = new_merchant_transactions['authorized_flag'].map({'Y': 1, 'N': 0})
new_merchant_transactions['category_1'] = new_merchant_transactions['category_1'].map({'Y': 1, 'N': 0})

In [ ]:
new_merchant_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,price
0,1,C_ID_415bb3a509,107,0,1.0,B,307,M_ID_b0c793002c,1,-0.557617,2018-03-11 14:57:36,1.0,9,19,-0.552096
1,1,C_ID_415bb3a509,140,0,1.0,B,307,M_ID_88920c89e8,1,-0.569336,2018-03-19 18:53:37,1.0,9,19,-0.563699
2,1,C_ID_415bb3a509,330,0,1.0,B,507,M_ID_ad5237ef6b,2,-0.551270,2018-04-26 14:08:44,1.0,9,14,-0.545811
3,1,C_ID_415bb3a509,-1,1,1.0,B,661,M_ID_9e84cda3b1,1,-0.671875,2018-03-07 09:43:21,1.0,-1,8,-0.665223
4,1,C_ID_ef55cf8d4b,-1,1,1.0,B,166,M_ID_3c86fa3831,1,-0.659668,2018-03-22 21:07:53,1.0,-1,29,-0.653137


In [ ]:
new_merchant_transactions.to_csv('New_Data_processed/new_transactions_processed.csv')

### Featurization using purchase date

Referring the EDA kernel on dates: https://www.kaggle.com/denzo123/a-closer-look-at-date-variables

In [ ]:
new_merchant_transactions = reduce_mem_usage(pd.read_csv('New_Data_processed/new_transactions_processed.csv', index_col=0))

Mem. usage decreased to 1582.53 Mb (52.5% reduction)


In [ ]:
new_merchant_transactions = new_merchant_transactions[['card_id', 'purchase_date', 'month_lag']]

In [ ]:
pd.to_datetime(new_merchant_transactions['purchase_date'], format='%Y-%m-%d %H:%M:%S').dt.year.value_counts()

2018    1659548
2017     303483
Name: purchase_date, dtype: int64

So our data contains transactions from two years: 2017 and 2018.

In [ ]:
new_merchant_transactions['purchase_date'] = pd.to_datetime(new_merchant_transactions['purchase_date'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
# Get the day of the week for the purchase date
def get_weekday(data):
  return data.dt.dayofweek

# Return 1 if the purchase date is on a weekend
def is_weekend(day):
  if day == 5 or day == 6:
    return 1
  else:
    return 0

#Return day of purchase
def get_day(date_obj):
  return date_obj.dt.day

#Return week of the year of purchase
def get_week_of_year(date_obj):
  return date_obj.dt.weekofyear

#Return hour of purchase
def get_hour(date_obj):
  return date_obj.dt.hour

#Return month of purchase date
def get_purchase_month(data):
  return data.dt.month

#Return the month phase during the purchase
#Eg. Early in the month, mid-month or at the end of the month
def get_time_of_month(date):
  if date.day <=10:
    return "Early"
  elif date.day > 10 and date.day <= 20:
    return "Mid"
  else:
    return "End"
  

# Time of the day during purchase
#Eg. Morning, Afternoon, Evening, Night
def get_time_of_day(time):
  if time.hour >= 4 and time.hour < 12:
    return "Morning"
  elif time.hour >= 12 and time.hour < 17:
    return "Afternoon"
  elif time.hour >= 17 and time.hour < 22:
    return "Evening"
  else:
    return "Night"
    

# Returns 1 if the purchase was made on a holiday(Saturdays and sundays excluded)
# Google Search : list of holidays in brazil 2017 and 2018
def get_isholiday(date):
  holiday_list=[
            '01-01-17', '14-02-17', '28-08-17', '14-04-17', '16-04-17', '21-04-17',
            '01-05-17', '15-06-17', '07-09-17', '12-10-17', '02-11-17', '15-11-17', 
            '24-12-17', '25-12-17', '31-12-17',
            '01-01-18', '14-02-18', '28-08-18', '14-04-18', '16-04-18', '21-04-18',
            '01-05-18', '15-06-18', '07-09-18', '12-10-18', '02-11-18', '15-11-18', 
            '24-12-18', '25-12-18', '31-12-18'
  ]
  date = date.strftime(format='%d-%m-%y') 
  if date in holiday_list:
    return 1
  else:
    return 0

In [ ]:
new_merchant_transactions.head()

,card_id,purchase_date,month_lag
0,C_ID_415bb3a509,2018-03-11 14:57:36,1
1,C_ID_415bb3a509,2018-03-19 18:53:37,1
2,C_ID_415bb3a509,2018-04-26 14:08:44,2
3,C_ID_415bb3a509,2018-03-07 09:43:21,1
4,C_ID_ef55cf8d4b,2018-03-22 21:07:53,1


In [ ]:
new_merchant_transactions['weekday'] = get_weekday(new_merchant_transactions['purchase_date'])
new_merchant_transactions['is_weekend'] = new_merchant_transactions['weekday'].apply(lambda day: is_weekend(day))
new_merchant_transactions['purchase_month'] = get_purchase_month(new_merchant_transactions['purchase_date'])
new_merchant_transactions['purchase_day'] = get_day(new_merchant_transactions['purchase_date'])
new_merchant_transactions['week_of_year'] = get_week_of_year(new_merchant_transactions['purchase_date'])
new_merchant_transactions['purchase_hour'] = get_hour(new_merchant_transactions['purchase_date'])
#historical_transactions['purchase_time_of_day'] = historical_transactions['purchase_date'].apply(lambda day: get_time_of_day(day))
#historical_transactions['purchase_month_time'] = historical_transactions['purchase_date'].apply(lambda date: get_time_of_month(date))
new_merchant_transactions['purchase_on_holiday'] = new_merchant_transactions['purchase_date'].apply(lambda date_obj: get_isholiday(date_obj))
new_merchant_transactions['purchase_date'] = new_merchant_transactions['purchase_date'].dt.date

In [ ]:
new_merchant_transactions.head()

,card_id,purchase_date,month_lag,weekday,is_weekend,purchase_month,purchase_day,week_of_year,purchase_hour,purchase_on_holiday
0,C_ID_415bb3a509,2018-03-11,1,6,1,3,11,10,14,0
1,C_ID_415bb3a509,2018-03-19,1,0,0,3,19,12,18,0
2,C_ID_415bb3a509,2018-04-26,2,3,0,4,26,17,14,0
3,C_ID_415bb3a509,2018-03-07,1,2,0,3,7,10,9,0
4,C_ID_ef55cf8d4b,2018-03-22,1,3,0,3,22,12,21,0


In [ ]:
new_merchant_transactions['month_diff'] = ((datetime.date.today() - new_merchant_transactions['purchase_date']).dt.days)//30
new_merchant_transactions['month_diff'] +=new_merchant_transactions['month_lag']
del new_merchant_transactions['month_lag']
new_merchant_transactions.head()

,card_id,purchase_date,weekday,is_weekend,purchase_month,purchase_day,week_of_year,purchase_hour,purchase_on_holiday,month_diff
0,C_ID_415bb3a509,2018-03-11,6,1,3,11,10,14,0,32
1,C_ID_415bb3a509,2018-03-19,0,0,3,19,12,18,0,31
2,C_ID_415bb3a509,2018-04-26,3,0,4,26,17,14,0,31
3,C_ID_415bb3a509,2018-03-07,2,0,3,7,10,9,0,32
4,C_ID_ef55cf8d4b,2018-03-22,3,0,3,22,12,21,0,31


In [ ]:
new_merchant_transactions.isna().any()

card_id                False
purchase_date          False
weekday                False
is_weekend             False
purchase_month         False
purchase_day           False
week_of_year           False
purchase_hour          False
purchase_on_holiday    False
month_diff             False
dtype: bool

Saving the dataframe due to memory constraints.

In [ ]:
new_merchant_transactions.to_csv('New_Data_processed/new_transactions_date_features.csv')

#### Aggregating:

In [ ]:
new_merchant_transactions = pd.read_csv('New_Data_processed/new_transactions_date_features.csv', index_col=0)

In [ ]:
new_merchant_transactions.head()

,card_id,purchase_date,weekday,is_weekend,purchase_month,purchase_day,week_of_year,purchase_hour,purchase_on_holiday,month_diff
0,C_ID_415bb3a509,2018-03-11,6,1,3,11,10,14,0,32
1,C_ID_415bb3a509,2018-03-19,0,0,3,19,12,18,0,31
2,C_ID_415bb3a509,2018-04-26,3,0,4,26,17,14,0,31
3,C_ID_415bb3a509,2018-03-07,2,0,3,7,10,9,0,32
4,C_ID_ef55cf8d4b,2018-03-22,3,0,3,22,12,21,0,31


In [ ]:
new_merchant_transactions['purchase_date'] = pd.to_datetime(new_merchant_transactions['purchase_date'])

In [ ]:
aggregations = {
    'is_weekend': ['sum', 'mean'],
    'purchase_on_holiday': ['sum', 'mean'],
    'weekday' : ['nunique', 'sum', 'mean'],
    'purchase_hour': ['nunique', 'mean', 'min', 'max'],
    'week_of_year': ['nunique', 'mean', 'min', 'max'],
    'month_diff': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'purchase_day': ['nunique', 'sum', 'min'],
    'purchase_date' : [np.ptp, 'min', 'max'],
    'purchase_month' : ['sum', 'mean', 'nunique']

}

aggregated_date_features = new_merchant_transactions.groupby('card_id').agg(aggregations)
aggregated_date_features.columns = ['new_transactions_'+'_'.join(col).strip() 
                           for col in aggregated_date_features.columns.values]

In [ ]:
aggregated_date_features.head()

,new_transactions_is_weekend_sum,new_transactions_is_weekend_mean,new_transactions_purchase_on_holiday_sum,new_transactions_purchase_on_holiday_mean,new_transactions_weekday_nunique,new_transactions_weekday_sum,new_transactions_weekday_mean,new_transactions_purchase_hour_nunique,new_transactions_purchase_hour_mean,new_transactions_purchase_hour_min,new_transactions_purchase_hour_max,new_transactions_week_of_year_nunique,new_transactions_week_of_year_mean,new_transactions_week_of_year_min,new_transactions_week_of_year_max,new_transactions_month_diff_sum,new_transactions_month_diff_mean,new_transactions_month_diff_min,new_transactions_month_diff_max,new_transactions_month_diff_var,new_transactions_month_diff_skew,new_transactions_purchase_day_nunique,new_transactions_purchase_day_sum,new_transactions_purchase_day_min,new_transactions_purchase_date_ptp,new_transactions_purchase_date_min,new_transactions_purchase_date_max,new_transactions_purchase_month_sum,new_transactions_purchase_month_mean,new_transactions_purchase_month_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,0,0.000000,0,0.000000,2,1,0.500000,2,13.500000,11,16,2,14.500000,14,15,64,32.000000,32,32,0.000000,NaN,2,12,3,6 days,2018-04-03,2018-04-09,8,4.000000,1
C_ID_0001238066,12,0.461538,3,0.115385,6,105,4.038462,16,15.192308,0,23,9,12.846154,9,18,817,31.423077,31,32,0.253846,0.330798,14,474,1,60 days,2018-03-01,2018-04-30,87,3.346154,2
C_ID_0001506ef0,0,0.000000,0,0.000000,2,7,3.500000,2,15.500000,9,22,2,11.500000,11,12,63,31.500000,31,32,0.500000,NaN,2,38,16,6 days,2018-03-16,2018-03-22,6,3.000000,1
C_ID_0001793786,14,0.451613,5,0.161290,6,111,3.580645,10,11.419355,0,21,6,48.387097,46,52,1093,35.258065,35,36,0.197849,1.162856,13,732,10,46 days,2017-11-15,2017-12-31,351,11.322581,2
C_ID_000183fdda,2,0.181818,0,0.000000,6,27,2.454545,8,15.454545,11,22,7,11.818182,9,18,349,31.727273,31,32,0.218182,-1.189373,9,129,2,59 days,2018-03-02,2018-04-30,36,3.272727,2


In [ ]:
aggregated_date_features['new_transactions_purchase_date_ptp'] = aggregated_date_features['new_transactions_purchase_date_ptp'].dt.days

In [ ]:
aggregated_date_features['new_transactions_purchase_date_max'].isna().any()

False

In [ ]:
d_now = datetime.datetime.today()
d_now

datetime.datetime(2020, 10, 1, 16, 1, 45, 634581)

In [ ]:
aggregated_date_features['new_transactions_purchase_date_max_diff_now'] = (d_now - aggregated_date_features['new_transactions_purchase_date_max']).dt.days

In [ ]:
aggregated_date_features['new_transactions_purchase_date_min_diff_now'] = (d_now - aggregated_date_features['new_transactions_purchase_date_min']).dt.days

In [ ]:
del aggregated_date_features['new_transactions_purchase_date_ptp']

In [ ]:
aggregated_date_features.head()

,new_transactions_is_weekend_sum,new_transactions_is_weekend_mean,new_transactions_purchase_on_holiday_sum,new_transactions_purchase_on_holiday_mean,new_transactions_weekday_nunique,new_transactions_weekday_sum,new_transactions_weekday_mean,new_transactions_purchase_hour_nunique,new_transactions_purchase_hour_mean,new_transactions_purchase_hour_min,new_transactions_purchase_hour_max,new_transactions_week_of_year_nunique,new_transactions_week_of_year_mean,new_transactions_week_of_year_min,new_transactions_week_of_year_max,new_transactions_month_diff_sum,new_transactions_month_diff_mean,new_transactions_month_diff_min,new_transactions_month_diff_max,new_transactions_month_diff_var,new_transactions_month_diff_skew,new_transactions_purchase_day_nunique,new_transactions_purchase_day_sum,new_transactions_purchase_day_min,new_transactions_purchase_date_min,new_transactions_purchase_date_max,new_transactions_purchase_month_sum,new_transactions_purchase_month_mean,new_transactions_purchase_month_nunique,new_transactions_purchase_date_max_diff_now,new_transactions_purchase_date_min_diff_now
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,0,0.000000,0,0.000000,2,1,0.500000,2,13.500000,11,16,2,14.500000,14,15,64,32.000000,32,32,0.000000,NaN,2,12,3,2018-04-03,2018-04-09,8,4.000000,1,906,912
C_ID_0001238066,12,0.461538,3,0.115385,6,105,4.038462,16,15.192308,0,23,9,12.846154,9,18,817,31.423077,31,32,0.253846,0.330798,14,474,1,2018-03-01,2018-04-30,87,3.346154,2,885,945
C_ID_0001506ef0,0,0.000000,0,0.000000,2,7,3.500000,2,15.500000,9,22,2,11.500000,11,12,63,31.500000,31,32,0.500000,NaN,2,38,16,2018-03-16,2018-03-22,6,3.000000,1,924,930
C_ID_0001793786,14,0.451613,5,0.161290,6,111,3.580645,10,11.419355,0,21,6,48.387097,46,52,1093,35.258065,35,36,0.197849,1.162856,13,732,10,2017-11-15,2017-12-31,351,11.322581,2,1005,1051
C_ID_000183fdda,2,0.181818,0,0.000000,6,27,2.454545,8,15.454545,11,22,7,11.818182,9,18,349,31.727273,31,32,0.218182,-1.189373,9,129,2,2018-03-02,2018-04-30,36,3.272727,2,885,944


In [ ]:
aggregated_date_features.columns[aggregated_date_features.isna().any()]

Index(['new_transactions_month_diff_var', 'new_transactions_month_diff_skew'], dtype='object')

In [ ]:
aggregated_date_features.to_csv('New_Data_processed/new_transactions_date_features_final.csv')

### Other Features from transactions


In [ ]:
new_merchant_transactions = reduce_mem_usage(pd.read_csv('New_Data_processed/new_transactions_processed.csv', index_col = 0))

Mem. usage decreased to 102.97 Mb (54.2% reduction)


In [ ]:
date_features = reduce_mem_usage(pd.read_csv('New_Data_processed/new_transactions_date_features.csv', index_col = 0))

Mem. usage decreased to 59.91 Mb (63.6% reduction)


In [ ]:
month_diff = date_features['month_diff']

In [ ]:
new_merchant_transactions['duration'] = new_merchant_transactions['purchase_amount']*month_diff
new_merchant_transactions['amount_month_ratio'] = new_merchant_transactions['purchase_amount']/month_diff
new_merchant_transactions['purchase_amount'] = new_merchant_transactions['purchase_amount'].apply(lambda x: min(x, 0.8))
new_merchant_transactions['installments'].replace([-1, 999], np.nan, inplace=True)
new_merchant_transactions['price'] = new_merchant_transactions['purchase_amount'] / new_merchant_transactions['installments']

In [ ]:
del month_diff,date_features

gc.collect()

0

In [ ]:
new_merchant_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,price,duration,amount_month_ratio
0,1,C_ID_415bb3a509,107,0,1.0,B,307,M_ID_b0c793002c,1,-0.557617,2018-03-11 14:57:36,1.0,9,19,-0.557617,-17.843750,-0.017426
1,1,C_ID_415bb3a509,140,0,1.0,B,307,M_ID_88920c89e8,1,-0.569336,2018-03-19 18:53:37,1.0,9,19,-0.569336,-17.649414,-0.018366
2,1,C_ID_415bb3a509,330,0,1.0,B,507,M_ID_ad5237ef6b,2,-0.551270,2018-04-26 14:08:44,1.0,9,14,-0.551270,-17.089355,-0.017783
3,1,C_ID_415bb3a509,-1,1,1.0,B,661,M_ID_9e84cda3b1,1,-0.671875,2018-03-07 09:43:21,1.0,-1,8,-0.671875,-21.500000,-0.020996
4,1,C_ID_ef55cf8d4b,-1,1,1.0,B,166,M_ID_3c86fa3831,1,-0.659668,2018-03-22 21:07:53,1.0,-1,29,-0.659668,-20.449707,-0.021280


In [ ]:
cat_aggregations = pd.DataFrame()
for col in ['category_2', 'category_3']:
  cat_aggregations[col + '_mean'] = new_merchant_transactions.groupby([col])['purchase_amount'].transform('mean')
  cat_aggregations[col + '_min'] = new_merchant_transactions.groupby([col])['purchase_amount'].transform('min')
  cat_aggregations[col + '_max'] = new_merchant_transactions.groupby([col])['purchase_amount'].transform('max')
  cat_aggregations[col + '_sum'] = new_merchant_transactions.groupby([col])['purchase_amount'].transform('sum')

In [ ]:
cat_aggregations['card_id'] = new_merchant_transactions['card_id']

In [ ]:
cat_aggregations.head()

,category_2_mean,category_2_min,category_2_max,category_2_sum,category_3_mean,category_3_min,category_3_max,category_3_sum,card_id
0,-0.584813,-0.74707,0.8,-684223.773218,-0.618676,-0.74707,0.8,-517323.217881,C_ID_415bb3a509
1,-0.584813,-0.74707,0.8,-684223.773218,-0.618676,-0.74707,0.8,-517323.217881,C_ID_415bb3a509
2,-0.584813,-0.74707,0.8,-684223.773218,-0.618676,-0.74707,0.8,-517323.217881,C_ID_415bb3a509
3,-0.584813,-0.74707,0.8,-684223.773218,-0.618676,-0.74707,0.8,-517323.217881,C_ID_415bb3a509
4,-0.584813,-0.74707,0.8,-684223.773218,-0.618676,-0.74707,0.8,-517323.217881,C_ID_ef55cf8d4b


In [ ]:
new_merchant_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,price,duration,amount_month_ratio
0,1,C_ID_415bb3a509,107,0,1.0,B,307,M_ID_b0c793002c,1,-0.557617,2018-03-11 14:57:36,1.0,9,19,-0.557617,-17.843750,-0.017426
1,1,C_ID_415bb3a509,140,0,1.0,B,307,M_ID_88920c89e8,1,-0.569336,2018-03-19 18:53:37,1.0,9,19,-0.569336,-17.649414,-0.018366
2,1,C_ID_415bb3a509,330,0,1.0,B,507,M_ID_ad5237ef6b,2,-0.551270,2018-04-26 14:08:44,1.0,9,14,-0.551270,-17.089355,-0.017783
3,1,C_ID_415bb3a509,-1,1,1.0,B,661,M_ID_9e84cda3b1,1,-0.671875,2018-03-07 09:43:21,1.0,-1,8,-0.671875,-21.500000,-0.020996
4,1,C_ID_ef55cf8d4b,-1,1,1.0,B,166,M_ID_3c86fa3831,1,-0.659668,2018-03-22 21:07:53,1.0,-1,29,-0.659668,-20.449707,-0.021280


In [ ]:
cat_aggregations.to_csv('New_Data_processed/new_transactions_category_aggregations_features.csv')

In [ ]:
del cat_aggregations
gc.collect()

24

In [ ]:
new_merchant_transactions.drop(columns=['purchase_date'], axis=1, inplace=True)

In [ ]:
new_merchant_transactions.to_csv('New_Data_processed/new_transactions_processed.csv')

Saving the dataframe due to memory constraints

#### Aggregating the category features generated using transform

In [ ]:
cat_aggregations = reduce_mem_usage(pd.read_csv('New_Data_processed/new_transactions_category_aggregations_features.csv'))

Mem. usage decreased to 59.91 Mb (60.0% reduction)


In [ ]:
aggregated = cat_aggregations.groupby('card_id').mean()

In [ ]:
aggregated.head()

,Unnamed: 0,category_2_mean,category_2_min,category_2_max,category_2_sum,category_3_mean,category_3_min,category_3_max,category_3_sum
card_id,,,,,,,,,
C_ID_00007093c1,1793298.5,-0.582520,-0.74707,0.799805,-426056.843750,-0.618652,-0.747070,0.799805,-517323.21875
C_ID_0001238066,38628.5,-0.584473,-0.74707,0.799805,-622638.562500,-0.553711,-0.747070,0.799805,-445363.65625
C_ID_0001506ef0,1443419.5,-0.580078,-0.74707,0.799805,-167889.921875,-0.612305,-0.747070,0.799805,-598762.43750
C_ID_0001793786,29154.0,-0.583008,-0.74707,0.799805,-417001.156250,-0.612305,-0.747070,0.799805,-598762.43750
C_ID_000183fdda,827141.0,-0.580078,-0.74707,0.799805,-167889.921875,-0.464600,-0.746582,0.799805,-347236.96875


In [ ]:
aggregated.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
columns = []
for column in aggregated.columns:
  columns.append('new_' + column)

In [ ]:
columns

['new_category_2_mean',
 'new_category_2_min',
 'new_category_2_max',
 'new_category_2_sum',
 'new_category_3_mean',
 'new_category_3_min',
 'new_category_3_max',
 'new_category_3_sum']

In [ ]:
aggregated.columns = columns

In [ ]:
aggregated.columns

Index(['new_category_2_mean', 'new_category_2_min', 'new_category_2_max',
       'new_category_2_sum', 'new_category_3_mean', 'new_category_3_min',
       'new_category_3_max', 'new_category_3_sum'],
      dtype='object')

In [ ]:
aggregated.to_csv('New_Data_processed/new_transactions_category_aggregated_final.csv')

#### Numerical Features aggregation

In [ ]:
new_merchant_transactions = reduce_mem_usage(pd.read_csv('New_Data_processed/new_transactions_processed.csv', index_col=0))

Mem. usage decreased to 110.45 Mb (56.6% reduction)


In [ ]:
new_merchant_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,category_2,state_id,subsector_id,price,duration,amount_month_ratio
0,1,C_ID_415bb3a509,107,0,1.0,B,307,M_ID_b0c793002c,1,-0.557617,1.0,9,19,-0.557617,-17.843750,-0.017426
1,1,C_ID_415bb3a509,140,0,1.0,B,307,M_ID_88920c89e8,1,-0.569336,1.0,9,19,-0.569336,-17.656250,-0.018372
2,1,C_ID_415bb3a509,330,0,1.0,B,507,M_ID_ad5237ef6b,2,-0.551270,1.0,9,14,-0.551270,-17.093750,-0.017776
3,1,C_ID_415bb3a509,-1,1,1.0,B,661,M_ID_9e84cda3b1,1,-0.671875,1.0,-1,8,-0.671875,-21.500000,-0.020996
4,1,C_ID_ef55cf8d4b,-1,1,1.0,B,166,M_ID_3c86fa3831,1,-0.659668,1.0,-1,29,-0.659668,-20.453125,-0.021286


#### Aggregating the columns based on card_id

In [ ]:
aggregations = {
    'authorized_flag' : ['sum', 'mean'],
    'category_1' : ['sum', 'mean'],
    'card_id': ['size'],
    'city_id' : ['nunique'],
    'state_id' : ['nunique'],
    'subsector_id' : ['nunique'],
    'merchant_category_id' : ['nunique'],
    'merchant_id': ['nunique'],
    'month_lag' : ['sum', 'mean', 'min', 'max', 'var'],
    'duration': ['mean', 'min', 'mean', 'max', 'var', 'skew'],
    'amount_month_ratio': ['mean', 'min', 'max', 'var', 'skew'],
    'installments' : ['sum', 'mean', 'min', 'max', 'var'],
    'purchase_amount' : ['sum', 'mean', 'min', 'max', 'var'],
    'price': ['sum', 'mean', 'min', 'max', 'var', 'skew']
}

aggregated_numerical_features = new_merchant_transactions.groupby('card_id').agg(aggregations)
aggregated_numerical_features.columns = ['new_transactions_'+'_'.join(col).strip() 
                           for col in aggregated_numerical_features.columns.values]

In [ ]:
aggregated_numerical_features.head()

,new_transactions_authorized_flag_sum,new_transactions_authorized_flag_mean,new_transactions_category_1_sum,new_transactions_category_1_mean,new_transactions_card_id_size,new_transactions_city_id_nunique,new_transactions_state_id_nunique,new_transactions_subsector_id_nunique,new_transactions_merchant_category_id_nunique,new_transactions_merchant_id_nunique,new_transactions_month_lag_sum,new_transactions_month_lag_mean,new_transactions_month_lag_min,new_transactions_month_lag_max,new_transactions_month_lag_var,new_transactions_duration_mean,new_transactions_duration_min,new_transactions_duration_mean,new_transactions_duration_max,new_transactions_duration_var,new_transactions_duration_skew,new_transactions_amount_month_ratio_mean,new_transactions_amount_month_ratio_min,new_transactions_amount_month_ratio_max,new_transactions_amount_month_ratio_var,new_transactions_amount_month_ratio_skew,new_transactions_installments_sum,new_transactions_installments_mean,new_transactions_installments_min,new_transactions_installments_max,new_transactions_installments_var,new_transactions_purchase_amount_sum,new_transactions_purchase_amount_mean,new_transactions_purchase_amount_min,new_transactions_purchase_amount_max,new_transactions_purchase_amount_var,new_transactions_price_sum,new_transactions_price_mean,new_transactions_price_min,new_transactions_price_max,new_transactions_price_var,new_transactions_price_skew
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,2,1,0,0.000000,2,2,2,2,2,2,4.0,2.000000,2,2,0.000000,-21.250000,-21.500000,-21.250000,-21.015625,0.117310,NaN,-0.020752,-0.020996,-0.020523,1.192093e-07,NaN,2.0,1.000000,1.0,1.0,0.000000,-1.328125,-0.664062,-0.671875,-0.656738,0.000115,-1.328613,-0.664307,-0.671875,-0.656738,0.000115,NaN
C_ID_0001238066,26,1,2,0.076923,26,8,4,9,15,25,35.0,1.346154,1,2,0.235385,-17.843750,-23.453125,-17.843750,-2.427734,29.312500,1.575195,-0.018280,-0.023895,-0.002525,3.099442e-05,1.541992,43.0,1.654297,0.0,10.0,4.156250,-14.851562,-0.571289,-0.740723,-0.078308,0.030075,-inf,-inf,-inf,-0.026103,NaN,NaN
C_ID_0001506ef0,2,1,0,0.000000,2,1,1,2,2,2,2.0,1.000000,1,1,0.000000,-22.437500,-22.687500,-22.437500,-22.171875,0.132935,NaN,-0.023346,-0.023605,-0.023071,1.192093e-07,NaN,0.0,0.000000,0.0,0.0,0.000000,-1.447266,-0.723633,-0.731934,-0.715332,0.000138,-inf,-inf,-inf,-inf,NaN,NaN
C_ID_0001793786,31,1,0,0.000000,31,7,5,14,21,31,41.0,1.322581,1,2,0.225806,-5.902344,-25.828125,-5.902344,28.000000,347.750000,0.883789,-0.004780,-0.021072,0.022858,2.312660e-04,0.879395,0.0,0.000000,0.0,0.0,0.000000,-5.207031,-0.167969,-0.737793,0.799805,0.283447,NaN,NaN,-inf,inf,NaN,NaN
C_ID_000183fdda,11,1,0,0.000000,11,2,2,6,9,11,14.0,1.272727,1,2,0.218182,-19.046875,-23.437500,-19.046875,-3.337891,34.437500,2.242188,-0.018845,-0.022888,-0.003473,3.260374e-05,2.308594,17.0,1.545898,0.0,4.0,1.272461,-6.589844,-0.599121,-0.732422,-0.107666,0.033447,-inf,-inf,-inf,-0.114929,NaN,NaN


In [ ]:
aggregated_numerical_features.to_csv('New_Data_processed/new_transactions_other_features_final.csv')

# Merchants File

#### Joining with the card_ids

In [ ]:
merchants = reduce_mem_usage(pd.read_csv('Data/merchants.csv'))
merchants.drop(columns=['merchant_category_id','subsector_id', 'city_id', 'state_id'], axis = 1, inplace=True)

Mem. usage decreased to 30.32 Mb (46.0% reduction)


In [ ]:
merged_merchants_with_cards = reduce_mem_usage(pd.read_csv('Data_processed/all_transactions.csv', index_col = 0))
merged_merchants_with_cards = merged_merchants_with_cards[['card_id', 'merchant_id']]

In [ ]:
merged_merchants_with_cards = pd.merge(merged_merchants_with_cards, merchants, on='merchant_id', how='left')

In [ ]:
merged_merchants_with_cards.head()

,card_id,merchant_id,merchant_group_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,category_2
0,C_ID_4e6213e9bc,M_ID_e020e9b302,35.0,23.468750,23.015625,N,A,A,1.08,1.082451,3.0,1.14,1.114135,6.0,1.19,1.156844,12.0,Y,1.0
1,C_ID_4e6213e9bc,M_ID_86ec983688,2084.0,1.092773,-0.057465,N,A,A,1.06,1.052071,3.0,1.06,1.058605,6.0,1.05,1.062087,12.0,Y,1.0
2,C_ID_4e6213e9bc,M_ID_979ed661fc,27369.0,0.021851,0.021851,N,C,C,0.98,0.974653,3.0,0.98,0.967058,6.0,0.97,0.956668,12.0,Y,1.0
3,C_ID_4e6213e9bc,M_ID_e6d5ae8ea6,24104.0,-0.057465,-0.057465,N,D,C,1.00,1.053443,3.0,0.88,0.897406,6.0,0.86,0.864394,12.0,Y,1.0
4,C_ID_4e6213e9bc,M_ID_e020e9b302,35.0,23.468750,23.015625,N,A,A,1.08,1.082451,3.0,1.14,1.114135,6.0,1.19,1.156844,12.0,Y,1.0


In [ ]:
merged_merchants_categorical_features = merged_merchants_with_cards[['card_id', 'merchant_group_id',
                                                                   'category_1', 'most_recent_sales_range',
                                                                   'most_recent_purchases_range', 'active_months_lag3',
                                                                   'active_months_lag6', 'active_months_lag12',
                                                                   'category_4', 'category_2',
                                                                   ]]
                                                                  
merged_merchants_numerical_features = merged_merchants_with_cards[['card_id', 'numerical_1', 'numerical_2',
                                                                   'avg_sales_lag3', 'avg_purchases_lag3',
                                                                   'avg_sales_lag6', 'avg_purchases_lag6',
                                                                   'avg_sales_lag12', 'avg_purchases_lag12',]]                                                                  

In [ ]:
merged_merchants_categorical_features.to_csv('Data_processed/merged_merchants_categorical_features.csv')
merged_merchants_numerical_features.to_csv('Data_processed/merged_merchants_numerical_features.csv')

#### Categorical Features

In [ ]:
merged_merchants_categorical_features = reduce_mem_usage(pd.read_csv('Data_processed/merged_merchants_categorical_features.csv', index_col=0))

Mem. usage decreased to 1849.86 Mb (31.8% reduction)


In [ ]:
merged_merchants_categorical_features.isna().any()

card_id                        False
merchant_group_id               True
category_1                      True
most_recent_sales_range         True
most_recent_purchases_range     True
active_months_lag3              True
active_months_lag6              True
active_months_lag12             True
category_4                      True
category_2                      True
dtype: bool

In [ ]:
merged_merchants_categorical_features['merchant_group_id'].fillna(merged_merchants_categorical_features['merchant_group_id'].mode()[0], inplace=True)
merged_merchants_categorical_features['category_1'].fillna(merged_merchants_categorical_features['category_1'].mode()[0], inplace=True)
merged_merchants_categorical_features['most_recent_sales_range'].fillna(merged_merchants_categorical_features['most_recent_sales_range'].mode()[0], inplace=True)
merged_merchants_categorical_features['most_recent_purchases_range'].fillna(merged_merchants_categorical_features['most_recent_purchases_range'].mode()[0], inplace=True)
merged_merchants_categorical_features['active_months_lag3'].fillna(merged_merchants_categorical_features['active_months_lag3'].mode()[0], inplace=True)
merged_merchants_categorical_features['active_months_lag6'].fillna(merged_merchants_categorical_features['active_months_lag6'].mode()[0], inplace=True)
merged_merchants_categorical_features['active_months_lag12'].fillna(merged_merchants_categorical_features['active_months_lag12'].mode()[0], inplace=True)
merged_merchants_categorical_features['category_4'].fillna(merged_merchants_categorical_features['category_4'].mode()[0], inplace=True)
merged_merchants_categorical_features['category_2'].fillna(merged_merchants_categorical_features['category_2'].mode()[0], inplace=True)

In [ ]:
merged_merchants_categorical_features.isna().any()

card_id                        False
merchant_group_id              False
category_1                     False
most_recent_sales_range        False
most_recent_purchases_range    False
active_months_lag3             False
active_months_lag6             False
active_months_lag12            False
category_4                     False
category_2                     False
dtype: bool

In [ ]:
merged_merchants_categorical_features.head()

,card_id,merchant_group_id,category_1,most_recent_sales_range,most_recent_purchases_range,active_months_lag3,active_months_lag6,active_months_lag12,category_4,category_2
0,C_ID_4e6213e9bc,35.0,N,A,A,3.0,6.0,12.0,Y,1.0
1,C_ID_4e6213e9bc,2084.0,N,A,A,3.0,6.0,12.0,Y,1.0
2,C_ID_4e6213e9bc,27369.0,N,C,C,3.0,6.0,12.0,Y,1.0
3,C_ID_4e6213e9bc,24104.0,N,D,C,3.0,6.0,12.0,Y,1.0
4,C_ID_4e6213e9bc,35.0,N,A,A,3.0,6.0,12.0,Y,1.0


In [ ]:
merged_merchants_categorical_features['category_1'] = merged_merchants_categorical_features['category_1'].map({
    'Y': 1,
    'N': 0
})
merged_merchants_categorical_features['category_4'] = merged_merchants_categorical_features['category_4'].map({
    'Y': 1,
    'N': 0
})

In [ ]:
merged_merchants_categorical_features = pd.get_dummies(merged_merchants_categorical_features,columns=[
            'most_recent_sales_range','most_recent_purchases_range', 'active_months_lag3', 'active_months_lag6',
            'active_months_lag12', 'category_2'
])

In [ ]:
merged_merchants_categorical_features.head()

,card_id,merchant_group_id,category_1,category_4,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E,active_months_lag3_1.0,active_months_lag3_2.0,active_months_lag3_3.0,active_months_lag6_1.0,active_months_lag6_2.0,active_months_lag6_3.0,active_months_lag6_4.0,active_months_lag6_5.0,active_months_lag6_6.0,active_months_lag12_1.0,active_months_lag12_2.0,active_months_lag12_3.0,active_months_lag12_4.0,active_months_lag12_5.0,active_months_lag12_6.0,active_months_lag12_7.0,active_months_lag12_8.0,active_months_lag12_9.0,active_months_lag12_10.0,active_months_lag12_11.0,active_months_lag12_12.0,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0
0,C_ID_4e6213e9bc,35.0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
1,C_ID_4e6213e9bc,2084.0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
2,C_ID_4e6213e9bc,27369.0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
3,C_ID_4e6213e9bc,24104.0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
4,C_ID_4e6213e9bc,35.0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0


In [ ]:
aggregations = {
    'merchant_group_id' : ['nunique'],
    'category_1' : ['sum', 'mean', 'std'],
    'category_4' :['sum', 'mean', 'std'],
    'category_2_1.0' :['sum', 'mean', 'std'],
    'category_2_2.0' :['sum', 'mean', 'std'],
    'category_2_3.0' :['sum', 'mean', 'std'],
    'category_2_4.0' :['sum', 'mean', 'std'],
    'category_2_5.0' :['sum', 'mean', 'std'],
    'most_recent_sales_range_A' :['sum', 'mean', 'std'],
    'most_recent_sales_range_B' :['sum', 'mean', 'std'],
    'most_recent_sales_range_C' :['sum', 'mean', 'std'],
    'most_recent_sales_range_D' :['sum', 'mean', 'std'],
    'most_recent_sales_range_E' :['sum', 'mean', 'std'],
    'most_recent_purchases_range_A' :['sum', 'mean', 'std'],
    'most_recent_purchases_range_B' :['sum', 'mean', 'std'],
    'most_recent_purchases_range_C' :['sum', 'mean', 'std'],
    'most_recent_purchases_range_D' :['sum', 'mean', 'std'],
    'most_recent_purchases_range_E' :['sum', 'mean', 'std'],
    'active_months_lag3_1.0' :['sum', 'mean', 'std'],
    'active_months_lag3_2.0' :['sum', 'mean', 'std'],
    'active_months_lag3_3.0' :['sum', 'mean', 'std'],
    'active_months_lag6_1.0' :['sum', 'mean', 'std'],
    'active_months_lag6_2.0' :['sum', 'mean', 'std'],
    'active_months_lag6_3.0' :['sum', 'mean', 'std'],
    'active_months_lag6_4.0' :['sum', 'mean', 'std'],
    'active_months_lag6_5.0' :['sum', 'mean', 'std'],
    'active_months_lag6_6.0' :['sum', 'mean', 'std'],
    'active_months_lag12_1.0' :['sum', 'mean', 'std'],
    'active_months_lag12_2.0' :['sum', 'mean', 'std'],
    'active_months_lag12_3.0' :['sum', 'mean', 'std'],
    'active_months_lag12_4.0' :['sum', 'mean', 'std'],
    'active_months_lag12_5.0' :['sum', 'mean', 'std'],
    'active_months_lag12_6.0' :['sum', 'mean', 'std'],
    'active_months_lag12_7.0' :['sum', 'mean', 'std'],
    'active_months_lag12_8.0' :['sum', 'mean', 'std'],
    'active_months_lag12_9.0' :['sum', 'mean', 'std'],
    'active_months_lag12_10.0' :['sum', 'mean', 'std'],
    'active_months_lag12_11.0' :['sum', 'mean', 'std'],
    'active_months_lag12_12.0' :['sum', 'mean', 'std']
}

merchant_features = merged_merchants_categorical_features.groupby('card_id').agg(aggregations)
merchant_features.columns = ['merchant_'+'_'.join(col).strip() 
                           for col in merchant_features.columns.values]

In [ ]:
merchant_features.head()

,merchant_merchant_group_id_nunique,merchant_category_1_sum,merchant_category_1_mean,merchant_category_1_std,merchant_category_4_sum,merchant_category_4_mean,merchant_category_4_std,merchant_category_2_1.0_sum,merchant_category_2_1.0_mean,merchant_category_2_1.0_std,merchant_category_2_2.0_sum,merchant_category_2_2.0_mean,merchant_category_2_2.0_std,merchant_category_2_3.0_sum,merchant_category_2_3.0_mean,merchant_category_2_3.0_std,merchant_category_2_4.0_sum,merchant_category_2_4.0_mean,merchant_category_2_4.0_std,merchant_category_2_5.0_sum,merchant_category_2_5.0_mean,merchant_category_2_5.0_std,merchant_most_recent_sales_range_A_sum,merchant_most_recent_sales_range_A_mean,merchant_most_recent_sales_range_A_std,merchant_most_recent_sales_range_B_sum,merchant_most_recent_sales_range_B_mean,merchant_most_recent_sales_range_B_std,merchant_most_recent_sales_range_C_sum,merchant_most_recent_sales_range_C_mean,merchant_most_recent_sales_range_C_std,merchant_most_recent_sales_range_D_sum,merchant_most_recent_sales_range_D_mean,merchant_most_recent_sales_range_D_std,merchant_most_recent_sales_range_E_sum,merchant_most_recent_sales_range_E_mean,merchant_most_recent_sales_range_E_std,merchant_most_recent_purchases_range_A_sum,merchant_most_recent_purchases_range_A_mean,merchant_most_recent_purchases_range_A_std,...,merchant_active_months_lag6_5.0_std,merchant_active_months_lag6_6.0_sum,merchant_active_months_lag6_6.0_mean,merchant_active_months_lag6_6.0_std,merchant_active_months_lag12_1.0_sum,merchant_active_months_lag12_1.0_mean,merchant_active_months_lag12_1.0_std,merchant_active_months_lag12_2.0_sum,merchant_active_months_lag12_2.0_mean,merchant_active_months_lag12_2.0_std,merchant_active_months_lag12_3.0_sum,merchant_active_months_lag12_3.0_mean,merchant_active_months_lag12_3.0_std,merchant_active_months_lag12_4.0_sum,merchant_active_months_lag12_4.0_mean,merchant_active_months_lag12_4.0_std,merchant_active_months_lag12_5.0_sum,merchant_active_months_lag12_5.0_mean,merchant_active_months_lag12_5.0_std,merchant_active_months_lag12_6.0_sum,merchant_active_months_lag12_6.0_mean,merchant_active_months_lag12_6.0_std,merchant_active_months_lag12_7.0_sum,merchant_active_months_lag12_7.0_mean,merchant_active_months_lag12_7.0_std,merchant_active_months_lag12_8.0_sum,merchant_active_months_lag12_8.0_mean,merchant_active_months_lag12_8.0_std,merchant_active_months_lag12_9.0_sum,merchant_active_months_lag12_9.0_mean,merchant_active_months_lag12_9.0_std,merchant_active_months_lag12_10.0_sum,merchant_active_months_lag12_10.0_mean,merchant_active_months_lag12_10.0_std,merchant_active_months_lag12_11.0_sum,merchant_active_months_lag12_11.0_mean,merchant_active_months_lag12_11.0_std,merchant_active_months_lag12_12.0_sum,merchant_active_months_lag12_12.0_mean,merchant_active_months_lag12_12.0_std
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,15,30,0.197368,0.399328,52,0.342105,0.475983,30.0,0.197368,0.399328,0.0,0.000000,0.000000,122.0,0.802632,0.399328,0.0,0.0,0.0,0.0,0.00,0.000000,52.0,0.342105,0.475983,12.0,0.078947,0.270548,14.0,0.092105,0.290130,61.0,0.401316,0.491785,13.0,0.085526,0.280588,45.0,0.296053,0.458024,...,0.0,152.0,1.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,1.0,0.006579,0.081111,0,0.0,0.0,4,0.026316,0.160602,0,0.000000,0.000000,0,0.000000,0.000000,147.0,0.967105,0.178951
C_ID_0001238066,66,11,0.073333,0.261556,31,0.206667,0.406271,129.0,0.860000,0.348149,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,21.0,0.14,0.348149,23.0,0.153333,0.361516,24.0,0.160000,0.367834,72.0,0.480000,0.501274,23.0,0.153333,0.361516,8.0,0.053333,0.225450,24.0,0.160000,0.367834,...,0.0,150.0,1.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,1.0,0.006667,0.081650,0,0.0,0.0,0,0.000000,0.000000,1,0.006667,0.081650,1,0.006667,0.081650,147.0,0.980000,0.140469
C_ID_0001506ef0,19,4,0.057143,0.233791,69,0.985714,0.119523,5.0,0.071429,0.259399,0.0,0.000

In [ ]:
merchant_features.to_csv('New_Data_processed/merchant_categorical_features_final.csv')

#### Numerical Features

In [ ]:
merged_merchants_numerical_features = reduce_mem_usage(pd.read_csv('Data_processed/merged_merchants_numerical_features.csv', index_col=0))

Mem. usage decreased to 1726.54 Mb (30.0% reduction)


In [ ]:
merged_merchants_numerical_features.isna().any()

card_id                False
numerical_1             True
numerical_2             True
avg_sales_lag3          True
avg_purchases_lag3      True
avg_sales_lag6          True
avg_purchases_lag6      True
avg_sales_lag12         True
avg_purchases_lag12     True
dtype: bool

In [ ]:
merged_merchants_numerical_features['numerical_1'].fillna(merged_merchants_numerical_features['numerical_1'].mode()[0], inplace=True)
merged_merchants_numerical_features['numerical_2'].fillna(merged_merchants_numerical_features['numerical_2'].mode()[0], inplace=True)
merged_merchants_numerical_features['avg_sales_lag3'].fillna(merged_merchants_numerical_features['avg_sales_lag3'].mean(), inplace=True)
merged_merchants_numerical_features['avg_purchases_lag3'].fillna(merged_merchants_numerical_features['avg_purchases_lag3'].mean(), inplace=True)
merged_merchants_numerical_features['avg_sales_lag6'].fillna(merged_merchants_numerical_features['avg_sales_lag6'].mean(), inplace=True)
merged_merchants_numerical_features['avg_purchases_lag6'].fillna(merged_merchants_numerical_features['avg_purchases_lag6'].mean(), inplace=True)
merged_merchants_numerical_features['avg_sales_lag12'].fillna(merged_merchants_numerical_features['avg_sales_lag12'].mean(), inplace=True)
merged_merchants_numerical_features['avg_purchases_lag12'].fillna(merged_merchants_numerical_features['avg_purchases_lag12'].mean(), inplace=True)

In [ ]:
merged_merchants_numerical_features.isna().any()

card_id                False
numerical_1            False
numerical_2            False
avg_sales_lag3         False
avg_purchases_lag3     False
avg_sales_lag6         False
avg_purchases_lag6     False
avg_sales_lag12        False
avg_purchases_lag12    False
dtype: bool

In [ ]:
merged_merchants_numerical_features.head()

,card_id,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,avg_sales_lag6,avg_purchases_lag6,avg_sales_lag12,avg_purchases_lag12
0,C_ID_4e6213e9bc,23.468750,23.015625,1.08,1.082451,1.14,1.114135,1.19,1.156844
1,C_ID_4e6213e9bc,1.092773,-0.057465,1.06,1.052071,1.06,1.058605,1.05,1.062087
2,C_ID_4e6213e9bc,0.021851,0.021851,0.98,0.974653,0.98,0.967058,0.97,0.956668
3,C_ID_4e6213e9bc,-0.057465,-0.057465,1.00,1.053443,0.88,0.897406,0.86,0.864394
4,C_ID_4e6213e9bc,23.468750,23.015625,1.08,1.082451,1.14,1.114135,1.19,1.156844


In [ ]:
aggregations = {
    'numerical_1': ['sum', 'mean', 'min', 'max', 'std'],
    'numerical_2': ['sum', 'mean', 'min', 'max', 'std'],
    'avg_sales_lag3': ['sum', 'mean', 'std'],
    'avg_purchases_lag3': ['sum', 'mean', 'std'],
    'avg_sales_lag6': ['sum', 'mean', 'std'],
    'avg_purchases_lag6': ['sum', 'mean', 'std'],
    'avg_sales_lag12': ['sum', 'mean', 'std'],
    'avg_purchases_lag12': ['sum', 'mean', 'std'],
}

merchant_features = merged_merchants_numerical_features.groupby('card_id').agg(aggregations)
merchant_features.columns = ['merchant_'+'_'.join(col).strip() 
                           for col in merchant_features.columns.values]

In [ ]:
merchant_features.head()

,merchant_numerical_1_sum,merchant_numerical_1_mean,merchant_numerical_1_min,merchant_numerical_1_max,merchant_numerical_1_std,merchant_numerical_2_sum,merchant_numerical_2_mean,merchant_numerical_2_min,merchant_numerical_2_max,merchant_numerical_2_std,merchant_avg_sales_lag3_sum,merchant_avg_sales_lag3_mean,merchant_avg_sales_lag3_std,merchant_avg_purchases_lag3_sum,merchant_avg_purchases_lag3_mean,merchant_avg_purchases_lag3_std,merchant_avg_sales_lag6_sum,merchant_avg_sales_lag6_mean,merchant_avg_sales_lag6_std,merchant_avg_purchases_lag6_sum,merchant_avg_purchases_lag6_mean,merchant_avg_purchases_lag6_std,merchant_avg_sales_lag12_sum,merchant_avg_sales_lag12_mean,merchant_avg_sales_lag12_std,merchant_avg_purchases_lag12_sum,merchant_avg_purchases_lag12_mean,merchant_avg_purchases_lag12_std
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,270.0000,1.776367,-0.057465,15.859375,4.375000,267.000000,1.756836,-0.057465,15.828125,4.371094,152.680008,1.004474,0.498170,163.049539,1.072694,0.954117,161.149994,1.060197,0.616332,174.243906,1.146341,1.218933,169.599991,1.115789,0.689795,184.000034,1.210527,1.333689
C_ID_0001238066,1324.0000,8.828125,-0.057465,129.375000,30.109375,1312.000000,8.750000,-0.057465,128.500000,29.890625,300.223114,2.001487,10.004545,inf,inf,NaN,452.240326,3.014935,15.250711,inf,inf,NaN,474.727448,3.164850,16.183575,inf,inf,NaN
C_ID_0001506ef0,65.4375,0.935059,-0.057465,15.390625,2.554688,56.406250,0.805664,-0.057465,15.109375,2.548828,205.743134,2.939188,14.543916,inf,inf,NaN,202.580322,2.894005,13.702086,inf,inf,NaN,194.587448,2.779821,12.595557,inf,inf,NaN
C_ID_0001793786,287.2500,1.134766,-0.057465,117.812500,8.179688,275.500000,1.088867,-0.057465,116.875000,8.109375,343.700012,1.358498,1.971153,436.895288,1.726859,5.741965,582.750000,2.303360,10.823874,1145.928796,4.529363,35.684775,610.869995,2.414506,11.507896,1192.489052,4.713395,37.842564
C_ID_000183fdda,32.5000,0.203125,-0.057465,4.523438,0.733398,25.328125,0.158325,-0.057465,4.511719,0.728027,216.000000,1.350000,1.347978,248.187403,1.551171,2.224224,535.640015,3.347750,16.416605,1261.420670,7.883879,56.091244,571.169983,3.569812,18.553192,1358.721346,8.492008,61.670996


In [ ]:
merchant_features.to_csv('New_Data_processed/merchant_numerical_features_final.csv')

# Merging all features together

In [ ]:
train = reduce_mem_usage(pd.read_csv('New_Data_processed/processed_train.csv', index_col=0))
test = reduce_mem_usage(pd.read_csv('New_Data_processed/processed_test.csv', index_col=0))

Mem. usage decreased to 13.48 Mb (67.6% reduction)
Mem. usage decreased to  8.84 Mb (63.9% reduction)


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201917 entries, 0 to 201916
Data columns (total 26 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   first_active_month                  201917 non-null  object 
 1   card_id                             201917 non-null  object 
 2   feature_1                           201917 non-null  float16
 3   feature_2                           201917 non-null  float16
 4   feature_3                           201917 non-null  float16
 5   target                              201917 non-null  float16
 6   quarter_first_active_month          201917 non-null  int8   
 7   first_active_month_diff_from_today  201917 non-null  int16  
 8   rare_datapoints                     201917 non-null  int8   
 9   cat_time_feature1                   201917 non-null  float16
 10  cat_time_feature2                   201917 non-null  float16
 11  cat_time_feature3         

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123623 entries, 0 to 123622
Data columns (total 25 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   first_active_month                  123623 non-null  object 
 1   card_id                             123623 non-null  object 
 2   feature_1                           123623 non-null  float16
 3   feature_2                           123623 non-null  float16
 4   feature_3                           123623 non-null  float16
 5   quarter_first_active_month          123623 non-null  int8   
 6   first_active_month_diff_from_today  123623 non-null  int16  
 7   target                              0 non-null       float64
 8   cat_time_feature1                   123623 non-null  float16
 9   cat_time_feature2                   123623 non-null  float16
 10  cat_time_feature3                   123623 non-null  float16
 11  cat_time_ratio1           

### Merging Transaction Features

In [ ]:
to_join = reduce_mem_usage(pd.read_csv('New_Data_processed/historical_transactions_date_features_final.csv', index_col=0))
train = pd.merge(train, to_join, on='card_id', how='left')
train.head()

Mem. usage decreased to 23.28 Mb (70.7% reduction)


,first_active_month,card_id,feature_1,feature_2,feature_3,target,quarter_first_active_month,first_active_month_diff_from_today,rare_datapoints,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,transactions_week_of_year_max,transactions_month_diff_sum,transactions_month_diff_mean,transactions_month_diff_min,transactions_month_diff_max,transactions_month_diff_var,transactions_month_diff_skew,transactions_purchase_day_nunique,transactions_purchase_day_sum,transactions_purchase_day_min,transactions_purchase_date_min,transactions_purchase_date_max,transactions_purchase_month_sum,transactions_purchase_month_mean,transactions_purchase_month_nunique,transactions_purchase_date_max_diff_now,transactions_purchase_date_min_diff_now
0,2017-06-01,C_ID_92a2005557,0.013145,0.008751,0.011429,-0.820312,2,1215,0,15.968750,10.632812,13.882812,0.000011,0.000007,0.000009,0.033325,0.011108,0.013145,0.008751,0.002214,15.968750,0.000011,10.632812,0.000007,13.882812,0.000009,90,0.346191,8,0.030762,7,835,3.210938,23,13.312500,0,23,35,33.06250,1,52,8184,31.484375,31,32,0.250488,0.092957,31,4033,1,2017-06-27,2018-02-25,2095,8.054688,9,947,1190
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011383,0.010284,0.392822,1,1366,0,14.632812,15.554688,14.046875,0.000008,0.000008,0.000008,0.032379,0.010796,0.011383,0.010284,0.000556,14.632812,0.000008,15.554688,0.000008,14.046875,0.000008,132,0.377197,17,0.048584,7,1177,3.363281,24,14.718750,0,23,50,25.21875,1,52,11340,32.406250,32,33,0.240723,0.409912,31,5833,1,2017-01-06,2018-01-31,2177,6.218750,12,972,1362
2,2016-08-01,C_ID_d639edf6cd,0.010612,0.008751,0.010284,0.687988,3,1519,0,16.109375,13.296875,15.617188,0.000007,0.000006,0.000007,0.029648,0.009880,0.010612,0.008751,0.000992,16.109375,0.000007,13.296875,0.000006,15.617188,0.000007,11,0.255859,3,0.069763,7,142,3.302734,14,17.906250,8,23,22,18.37500,2,49,1347,31.328125,31,32,0.224854,0.771484,19,831,2,2017-01-11,2018-02-27,196,4.558594,10,945,1357
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014168,0.010284,0.142456,3,1123,0,12.031250,15.906250,11.546875,0.000010,0.000013,0.000009,0.035156,0.011719,0.014168,0.010284,0.002129,12.031250,0.000010,15.906250,0.000013,11.546875,0.000009,11,0.142822,0,0.000000,7,215,2.792969,16,14.445312,0,23,20,32.00000,1,52,2417,31.390625,31,32,0.240967,0.461670,25,1299,2,2017-09-26,2018-02-28,596,7.742188,6,944,1099
4,2017-11-01,C_ID_cdbd2c0db2,0.008057,0.014168,0.010284,-0.159790,4,1062,0,8.554688,15.046875,10.921875,0.000008,0.000013,0.000010,0.032501,0.010834,0.014168,0.008057,0.003092,8.554688,0.000008,15.046875,0.000013,10.921875,0.000010,42,0.315674,2,0.015038,7,431,3.240234,22,13.046875,0,23,17,21.78125,1,52,4171,31.359375,31,32,0.232422,0.585938,30,2141,1,2017-11-12,2018-02-28,719,5.406250,4,944,1052


In [ ]:
test = pd.merge(test, to_join, on='card_id', how='left')
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,quarter_first_active_month,first_active_month_diff_from_today,target,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,transactions_week_of_year_max,transactions_month_diff_sum,transactions_month_diff_mean,transactions_month_diff_min,transactions_month_diff_max,transactions_month_diff_var,transactions_month_diff_skew,transactions_purchase_day_nunique,transactions_purchase_day_sum,transactions_purchase_day_min,transactions_purchase_date_min,transactions_purchase_date_max,transactions_purchase_month_sum,transactions_purchase_month_mean,transactions_purchase_month_nunique,transactions_purchase_date_max_diff_now,transactions_purchase_date_min_diff_now
0,2017-04-01,C_ID_0ab67a22ab,0.010483,0.014168,0.011429,2,1276,NaN,13.375000,18.078125,14.578125,0.000008,0.000011,0.000009,0.036072,0.012024,0.014168,0.010483,0.001915,13.375000,0.000008,18.078125,0.000011,14.578125,0.000009,12,0.176514,0,0.000000,7,190,2.794922,17,14.367188,0,23,24,34.531250,14,52,2274,33.437500,33,34,0.250244,0.242310,24,1112,1,2017-04-04,2017-12-29,569,8.367188,9,1005,1274
1,2017-01-01,C_ID_130fd0cbdd,0.010612,0.014168,0.010284,1,1366,NaN,14.492188,19.343750,14.046875,0.000008,0.000010,0.000008,0.035065,0.011688,0.014168,0.010284,0.002153,14.492188,0.000008,19.343750,0.000010,14.046875,0.000008,17,0.217896,4,0.051270,7,229,2.935547,18,14.421875,0,23,20,12.437500,2,21,2455,31.468750,31,32,0.252686,0.104736,27,1288,2,2017-01-13,2018-02-18,256,3.281250,5,954,1355
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011383,0.011429,3,1154,NaN,15.171875,13.140625,13.187500,0.000011,0.000010,0.000010,0.035950,0.011986,0.013145,0.011383,0.001004,15.171875,0.000011,13.140625,0.000010,13.187500,0.000010,0,0.000000,0,0.000000,4,34,2.615234,7,17.000000,13,20,7,29.000000,2,49,414,31.843750,31,32,0.140991,-2.179688,7,111,2,2017-08-25,2018-02-02,95,7.308594,6,970,1131
3,2017-12-01,C_ID_d27d835a9f,0.010612,0.011383,0.010284,4,1032,NaN,10.953125,11.750000,10.609375,0.000010,0.000011,0.000010,0.032288,0.010757,0.011383,0.010284,0.000566,10.953125,0.000010,11.750000,0.000011,10.609375,0.000010,7,0.269287,0,0.000000,7,64,2.460938,9,17.921875,8,22,6,31.109375,4,50,822,31.609375,31,32,0.246094,-0.503906,11,297,4,2017-12-04,2018-02-20,200,7.691406,3,952,1030
4,2015-12-01,C_ID_2b5e3df5c2,0.013145,0.011383,0.011429,4,1763,NaN,23.171875,20.078125,20.140625,0.000007,0.000006,0.000006,0.035950,0.011986,0.013145,0.011383,0.001004,23.171875,0.000007,20.078125,0.000006,20.140625,0.000006,21,0.190918,2,0.018188,7,265,2.408203,21,14.437500,0,23,34,18.843750,1,52,3471,31.546875,31,32,0.249268,-0.222534,27,1460,1,2017-01-03,2018-02-27,531,4.828125,12,945,1365


In [ ]:
to_join = reduce_mem_usage(pd.read_csv('New_Data_processed/new_transactions_date_features_final.csv', index_col=0))
train = pd.merge(train, to_join, on='card_id', how='left')
train.head()

Mem. usage decreased to 18.81 Mb (73.4% reduction)


,first_active_month,card_id,feature_1,feature_2,feature_3,target,quarter_first_active_month,first_active_month_diff_from_today,rare_datapoints,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,...,transactions_purchase_day_sum,transactions_purchase_day_min,transactions_purchase_date_min,transactions_purchase_date_max,transactions_purchase_month_sum,transactions_purchase_month_mean,transactions_purchase_month_nunique,transactions_purchase_date_max_diff_now,transactions_purchase_date_min_diff_now,new_transactions_is_weekend_sum,new_transactions_is_weekend_mean,new_transactions_purchase_on_holiday_sum,new_transactions_purchase_on_holiday_mean,new_transactions_weekday_nunique,new_transactions_weekday_sum,new_transactions_weekday_mean,new_transactions_purchase_hour_nunique,new_transactions_purchase_hour_mean,new_transactions_purchase_hour_min,new_transactions_purchase_hour_max,new_transactions_week_of_year_nunique,new_transactions_week_of_year_mean,new_transactions_week_of_year_min,new_transactions_week_of_year_max,new_transactions_month_diff_sum,new_transactions_month_diff_mean,new_transactions_month_diff_min,new_transactions_month_diff_max,new_transactions_month_diff_var,new_transactions_month_diff_skew,new_transactions_purchase_day_nunique,new_transactions_purchase_day_sum,new_transactions_purchase_day_min,new_transactions_purchase_date_min,new_transactions_purchase_date_max,new_transactions_purchase_month_sum,new_transactions_purchase_month_mean,new_transactions_purchase_month_nunique,new_transactions_purchase_date_max_diff_now,new_transactions_purchase_date_min_diff_now
0,2017-06-01,C_ID_92a2005557,0.013145,0.008751,0.011429,-0.820312,2,1215,0,15.968750,10.632812,13.882812,0.000011,0.000007,0.000009,0.033325,0.011108,0.013145,0.008751,0.002214,15.968750,0.000011,10.632812,0.000007,13.882812,0.000009,90,0.346191,8,0.030762,7,835,3.210938,23,13.312500,0,23,35,33.06250,1,...,4033,1,2017-06-27,2018-02-25,2095,8.054688,9,947,1190,6.0,0.260986,1.0,0.043488,7.0,72.0,3.130859,8.0,12.867188,8.0,16.0,7.0,13.304688,10.0,17.0,725.0,31.515625,31.0,32.0,0.260986,-0.093262,17.0,378.0,5.0,2018-03-05,2018-04-29,80.0,3.478516,2.0,886.0,941.0
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011383,0.010284,0.392822,1,1366,0,14.632812,15.554688,14.046875,0.000008,0.000008,0.000008,0.032379,0.010796,0.011383,0.010284,0.000556,14.632812,0.000008,15.554688,0.000008,14.046875,0.000008,132,0.377197,17,0.048584,7,1177,3.363281,24,14.718750,0,23,50,25.21875,1,...,5833,1,2017-01-06,2018-01-31,2177,6.218750,12,972,1362,0.0,0.000000,0.0,0.000000,4.0,9.0,1.500000,5.0,11.164062,6.0,17.0,4.0,9.000000,5.0,13.0,195.0,32.500000,32.0,33.0,0.300049,0.000000,4.0,81.0,1.0,2018-02-01,2018-03-30,15.0,2.500000,2.0,916.0,973.0
2,2016-08-01,C_ID_d639edf6cd,0.010612,0.008751,0.010284,0.687988,3,1519,0,16.109375,13.296875,15.617188,0.000007,0.000006,0.000007,0.029648,0.009880,0.010612,0.008751,0.000992,16.109375,0.000007,13.296875,0.000006,15.617188,0.000007,11,0.255859,3,0.069763,7,142,3.302734,14,17.906250,8,23,22,18.37500,2,...,831,2,2017-01-11,2018-02-27,196,4.558594,10,945,1357,1.0,1.000000,0.0,0.000000,1.0,5.0,5.000000,1.0,17.000000,17.0,17.0,1.0,17.000000,17.0,17.0,31.0,31.000000,31.0,31.0,NaN,NaN,1.0,28.0,28.0,2018-04-28,2018-04-28,4.0,4.000000,1.0,887.0,887.0
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014168,0.010284,0.142456,3,112

In [ ]:
test = pd.merge(test, to_join, on='card_id', how='left')
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,quarter_first_active_month,first_active_month_diff_from_today,target,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,transactions_week_of_year_max,...,transactions_purchase_day_sum,transactions_purchase_day_min,transactions_purchase_date_min,transactions_purchase_date_max,transactions_purchase_month_sum,transactions_purchase_month_mean,transactions_purchase_month_nunique,transactions_purchase_date_max_diff_now,transactions_purchase_date_min_diff_now,new_transactions_is_weekend_sum,new_transactions_is_weekend_mean,new_transactions_purchase_on_holiday_sum,new_transactions_purchase_on_holiday_mean,new_transactions_weekday_nunique,new_transactions_weekday_sum,new_transactions_weekday_mean,new_transactions_purchase_hour_nunique,new_transactions_purchase_hour_mean,new_transactions_purchase_hour_min,new_transactions_purchase_hour_max,new_transactions_week_of_year_nunique,new_transactions_week_of_year_mean,new_transactions_week_of_year_min,new_transactions_week_of_year_max,new_transactions_month_diff_sum,new_transactions_month_diff_mean,new_transactions_month_diff_min,new_transactions_month_diff_max,new_transactions_month_diff_var,new_transactions_month_diff_skew,new_transactions_purchase_day_nunique,new_transactions_purchase_day_sum,new_transactions_purchase_day_min,new_transactions_purchase_date_min,new_transactions_purchase_date_max,new_transactions_purchase_month_sum,new_transactions_purchase_month_mean,new_transactions_purchase_month_nunique,new_transactions_purchase_date_max_diff_now,new_transactions_purchase_date_min_diff_now
0,2017-04-01,C_ID_0ab67a22ab,0.010483,0.014168,0.011429,2,1276,NaN,13.375000,18.078125,14.578125,0.000008,0.000011,0.000009,0.036072,0.012024,0.014168,0.010483,0.001915,13.375000,0.000008,18.078125,0.000011,14.578125,0.000009,12,0.176514,0,0.000000,7,190,2.794922,17,14.367188,0,23,24,34.531250,14,52,...,1112,1,2017-04-04,2017-12-29,569,8.367188,9,1005,1274,1.0,0.333252,0.0,0.000000,3.0,11.0,3.666016,3.0,13.664062,9.0,19.0,3.0,7.000000,5.0,9.0,100.0,33.34375,33.0,34.0,0.333252,1.732422,3.0,47.0,3.0,2018-02-03,2018-02-28,6.0,2.000000,1.0,946.0,971.0
1,2017-01-01,C_ID_130fd0cbdd,0.010612,0.014168,0.010284,1,1366,NaN,14.492188,19.343750,14.046875,0.000008,0.000010,0.000008,0.035065,0.011688,0.014168,0.010284,0.002153,14.492188,0.000008,19.343750,0.000010,14.046875,0.000008,17,0.217896,4,0.051270,7,229,2.935547,18,14.421875,0,23,20,12.437500,2,21,...,1288,2,2017-01-13,2018-02-18,256,3.281250,5,954,1355,3.0,0.333252,1.0,0.111084,6.0,27.0,3.000000,8.0,15.218750,9.0,23.0,6.0,12.445312,9.0,16.0,286.0,31.78125,31.0,32.0,0.194458,-1.620117,7.0,102.0,3.0,2018-03-03,2018-04-20,31.0,3.445312,2.0,895.0,943.0
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011383,0.011429,3,1154,NaN,15.171875,13.140625,13.187500,0.000011,0.000010,0.000010,0.035950,0.011986,0.013145,0.011383,0.001004,15.171875,0.000011,13.140625,0.000010,13.187500,0.000010,0,0.000000,0,0.000000,4,34,2.615234,7,17.000000,13,20,7,29.000000,2,49,...,111,2,2017-08-25,2018-02-02,95,7.308594,6,970,1131,0.0,0.000000,0.0,0.000000,2.0,4.0,2.000000,2.0,13.500000,13.0,14.0,2.0,10.000000,9.0,11.0,64.0,32.00000,32.0,32.0,0.000000,NaN,2.0,14.0,1.0,2018-03-01,2018-03-13,6.0,3.000000,1.0,933.0,945.0
3,2017-12-01,C_ID_d27d835a9f,0.010612,0.011383,0.010284,4,1032,NaN,10.953125,1

In [ ]:
to_join = reduce_mem_usage(pd.read_csv('New_Data_processed/transactions_category_aggregated_final.csv', index_col=0))
train = pd.merge(train, to_join, on='card_id', how='left')
train.head()

Mem. usage decreased to  8.69 Mb (61.1% reduction)


,first_active_month,card_id,feature_1,feature_2,feature_3,target,quarter_first_active_month,first_active_month_diff_from_today,rare_datapoints,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,...,transactions_purchase_date_min_diff_now,new_transactions_is_weekend_sum,new_transactions_is_weekend_mean,new_transactions_purchase_on_holiday_sum,new_transactions_purchase_on_holiday_mean,new_transactions_weekday_nunique,new_transactions_weekday_sum,new_transactions_weekday_mean,new_transactions_purchase_hour_nunique,new_transactions_purchase_hour_mean,new_transactions_purchase_hour_min,new_transactions_purchase_hour_max,new_transactions_week_of_year_nunique,new_transactions_week_of_year_mean,new_transactions_week_of_year_min,new_transactions_week_of_year_max,new_transactions_month_diff_sum,new_transactions_month_diff_mean,new_transactions_month_diff_min,new_transactions_month_diff_max,new_transactions_month_diff_var,new_transactions_month_diff_skew,new_transactions_purchase_day_nunique,new_transactions_purchase_day_sum,new_transactions_purchase_day_min,new_transactions_purchase_date_min,new_transactions_purchase_date_max,new_transactions_purchase_month_sum,new_transactions_purchase_month_mean,new_transactions_purchase_month_nunique,new_transactions_purchase_date_max_diff_now,new_transactions_purchase_date_min_diff_now,category_2_mean,category_2_min,category_2_max,category_2_sum,category_3_mean,category_3_min,category_3_max,category_3_sum
0,2017-06-01,C_ID_92a2005557,0.013145,0.008751,0.011429,-0.820312,2,1215,0,15.968750,10.632812,13.882812,0.000011,0.000007,0.000009,0.033325,0.011108,0.013145,0.008751,0.002214,15.968750,0.000011,10.632812,0.000007,13.882812,0.000009,90,0.346191,8,0.030762,7,835,3.210938,23,13.312500,0,23,35,33.06250,1,...,1190,6.0,0.260986,1.0,0.043488,7.0,72.0,3.130859,8.0,12.867188,8.0,16.0,7.0,13.304688,10.0,17.0,725.0,31.515625,31.0,32.0,0.260986,-0.093262,17.0,378.0,5.0,2018-03-05,2018-04-29,80.0,3.478516,2.0,886.0,941.0,-0.606445,-0.74707,0.799805,-10717841.00,-0.639160,-0.74707,0.799805,-9929442.0
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011383,0.010284,0.392822,1,1366,0,14.632812,15.554688,14.046875,0.000008,0.000008,0.000008,0.032379,0.010796,0.011383,0.010284,0.000556,14.632812,0.000008,15.554688,0.000008,14.046875,0.000008,132,0.377197,17,0.048584,7,1177,3.363281,24,14.718750,0,23,50,25.21875,1,...,1362,0.0,0.000000,0.0,0.000000,4.0,9.0,1.500000,5.0,11.164062,6.0,17.0,4.0,9.000000,5.0,13.0,195.0,32.500000,32.0,33.0,0.300049,0.000000,4.0,81.0,1.0,2018-02-01,2018-03-30,15.0,2.500000,2.0,916.0,973.0,-0.606445,-0.74707,0.799805,-10816712.00,-0.533203,-0.74707,0.799805,-5854655.0
2,2016-08-01,C_ID_d639edf6cd,0.010612,0.008751,0.010284,0.687988,3,1519,0,16.109375,13.296875,15.617188,0.000007,0.000006,0.000007,0.029648,0.009880,0.010612,0.008751,0.000992,16.109375,0.000007,13.296875,0.000006,15.617188,0.000007,11,0.255859,3,0.069763,7,142,3.302734,14,17.906250,8,23,22,18.37500,2,...,1357,1.0,1.000000,0.0,0.000000,1.0,5.0,5.000000,1.0,17.000000,17.0,17.0,1.0,17.000000,17.0,17.0,31.0,31.000000,31.0,31.0,NaN,NaN,1.0,28.0,28.0,2018-04-28,2018-04-28,4.0,4.000000,1.0,887.0,887.0,-0.604004,-0.74707,0.799805,-3044996.75,-0.639648,-0.74707,0.799805,-9971130.0
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014168,0.010284,0.142456,3,1123,0,12.031250,15.906250,11.546875,0.000010,0.0000

In [ ]:
test = pd.merge(test, to_join, on='card_id', how='left')
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,quarter_first_active_month,first_active_month_diff_from_today,target,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,transactions_week_of_year_max,...,transactions_purchase_date_min_diff_now,new_transactions_is_weekend_sum,new_transactions_is_weekend_mean,new_transactions_purchase_on_holiday_sum,new_transactions_purchase_on_holiday_mean,new_transactions_weekday_nunique,new_transactions_weekday_sum,new_transactions_weekday_mean,new_transactions_purchase_hour_nunique,new_transactions_purchase_hour_mean,new_transactions_purchase_hour_min,new_transactions_purchase_hour_max,new_transactions_week_of_year_nunique,new_transactions_week_of_year_mean,new_transactions_week_of_year_min,new_transactions_week_of_year_max,new_transactions_month_diff_sum,new_transactions_month_diff_mean,new_transactions_month_diff_min,new_transactions_month_diff_max,new_transactions_month_diff_var,new_transactions_month_diff_skew,new_transactions_purchase_day_nunique,new_transactions_purchase_day_sum,new_transactions_purchase_day_min,new_transactions_purchase_date_min,new_transactions_purchase_date_max,new_transactions_purchase_month_sum,new_transactions_purchase_month_mean,new_transactions_purchase_month_nunique,new_transactions_purchase_date_max_diff_now,new_transactions_purchase_date_min_diff_now,category_2_mean,category_2_min,category_2_max,category_2_sum,category_3_mean,category_3_min,category_3_max,category_3_sum
0,2017-04-01,C_ID_0ab67a22ab,0.010483,0.014168,0.011429,2,1276,NaN,13.375000,18.078125,14.578125,0.000008,0.000011,0.000009,0.036072,0.012024,0.014168,0.010483,0.001915,13.375000,0.000008,18.078125,0.000011,14.578125,0.000009,12,0.176514,0,0.000000,7,190,2.794922,17,14.367188,0,23,24,34.531250,14,52,...,1274,1.0,0.333252,0.0,0.000000,3.0,11.0,3.666016,3.0,13.664062,9.0,19.0,3.0,7.000000,5.0,9.0,100.0,33.34375,33.0,34.0,0.333252,1.732422,3.0,47.0,3.0,2018-02-03,2018-02-28,6.0,2.000000,1.0,946.0,971.0,-0.606445,-0.74707,0.799805,-10816712.00,-0.481689,-0.74707,0.799805,-5024630.5
1,2017-01-01,C_ID_130fd0cbdd,0.010612,0.014168,0.010284,1,1366,NaN,14.492188,19.343750,14.046875,0.000008,0.000010,0.000008,0.035065,0.011688,0.014168,0.010284,0.002153,14.492188,0.000008,19.343750,0.000010,14.046875,0.000008,17,0.217896,4,0.051270,7,229,2.935547,18,14.421875,0,23,20,12.437500,2,21,...,1355,3.0,0.333252,1.0,0.111084,6.0,27.0,3.000000,8.0,15.218750,9.0,23.0,6.0,12.445312,9.0,16.0,286.0,31.78125,31.0,32.0,0.194458,-1.620117,7.0,102.0,3.0,2018-03-03,2018-04-20,31.0,3.445312,2.0,895.0,943.0,-0.608398,-0.74707,0.799805,-1839159.50,-0.610840,-0.74707,0.799805,-7084146.5
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011383,0.011429,3,1154,NaN,15.171875,13.140625,13.187500,0.000011,0.000010,0.000010,0.035950,0.011986,0.013145,0.011383,0.001004,15.171875,0.000011,13.140625,0.000010,13.187500,0.000010,0,0.000000,0,0.000000,4,34,2.615234,7,17.000000,13,20,7,29.000000,2,49,...,1131,0.0,0.000000,0.0,0.000000,2.0,4.0,2.000000,2.0,13.500000,13.0,14.0,2.0,10.000000,9.0,11.0,64.0,32.00000,32.0,32.0,0.000000,NaN,2.0,14.0,1.0,2018-03-01,2018-03-13,6.0,3.000000,1.0,933.0,945.0,-0.604004,-0.74707,0.799805,-3566176.75,-0.325928,-0.74707,0.799805,-3100319.0
3,2017-12-01,C_ID_d27d835a9f,0.010612,0.011383,0.010284,4,1032,NaN,10.953125,11.750000,10.609375,0.000010,0.000011,0.000010,

In [ ]:
to_join = reduce_mem_usage(pd.read_csv('New_Data_processed/new_transactions_category_aggregated_final.csv', index_col=0))
train = pd.merge(train, to_join, on='card_id', how='left')
train.head()

Mem. usage decreased to  7.74 Mb (61.1% reduction)


,first_active_month,card_id,feature_1,feature_2,feature_3,target,quarter_first_active_month,first_active_month_diff_from_today,rare_datapoints,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,...,new_transactions_purchase_hour_nunique,new_transactions_purchase_hour_mean,new_transactions_purchase_hour_min,new_transactions_purchase_hour_max,new_transactions_week_of_year_nunique,new_transactions_week_of_year_mean,new_transactions_week_of_year_min,new_transactions_week_of_year_max,new_transactions_month_diff_sum,new_transactions_month_diff_mean,new_transactions_month_diff_min,new_transactions_month_diff_max,new_transactions_month_diff_var,new_transactions_month_diff_skew,new_transactions_purchase_day_nunique,new_transactions_purchase_day_sum,new_transactions_purchase_day_min,new_transactions_purchase_date_min,new_transactions_purchase_date_max,new_transactions_purchase_month_sum,new_transactions_purchase_month_mean,new_transactions_purchase_month_nunique,new_transactions_purchase_date_max_diff_now,new_transactions_purchase_date_min_diff_now,category_2_mean,category_2_min,category_2_max,category_2_sum,category_3_mean,category_3_min,category_3_max,category_3_sum,new_category_2_mean,new_category_2_min,new_category_2_max,new_category_2_sum,new_category_3_mean,new_category_3_min,new_category_3_max,new_category_3_sum
0,2017-06-01,C_ID_92a2005557,0.013145,0.008751,0.011429,-0.820312,2,1215,0,15.968750,10.632812,13.882812,0.000011,0.000007,0.000009,0.033325,0.011108,0.013145,0.008751,0.002214,15.968750,0.000011,10.632812,0.000007,13.882812,0.000009,90,0.346191,8,0.030762,7,835,3.210938,23,13.312500,0,23,35,33.06250,1,...,8.0,12.867188,8.0,16.0,7.0,13.304688,10.0,17.0,725.0,31.515625,31.0,32.0,0.260986,-0.093262,17.0,378.0,5.0,2018-03-05,2018-04-29,80.0,3.478516,2.0,886.0,941.0,-0.606445,-0.74707,0.799805,-10717841.00,-0.639160,-0.74707,0.799805,-9929442.0,-0.584961,-0.74707,0.799805,-684223.750000,-0.612305,-0.74707,0.799805,-598762.43750
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011383,0.010284,0.392822,1,1366,0,14.632812,15.554688,14.046875,0.000008,0.000008,0.000008,0.032379,0.010796,0.011383,0.010284,0.000556,14.632812,0.000008,15.554688,0.000008,14.046875,0.000008,132,0.377197,17,0.048584,7,1177,3.363281,24,14.718750,0,23,50,25.21875,1,...,5.0,11.164062,6.0,17.0,4.0,9.000000,5.0,13.0,195.0,32.500000,32.0,33.0,0.300049,0.000000,4.0,81.0,1.0,2018-02-01,2018-03-30,15.0,2.500000,2.0,916.0,973.0,-0.606445,-0.74707,0.799805,-10816712.00,-0.533203,-0.74707,0.799805,-5854655.0,-0.584961,-0.74707,0.799805,-684223.750000,-0.618652,-0.74707,0.799805,-517323.21875
2,2016-08-01,C_ID_d639edf6cd,0.010612,0.008751,0.010284,0.687988,3,1519,0,16.109375,13.296875,15.617188,0.000007,0.000006,0.000007,0.029648,0.009880,0.010612,0.008751,0.000992,16.109375,0.000007,13.296875,0.000006,15.617188,0.000007,11,0.255859,3,0.069763,7,142,3.302734,14,17.906250,8,23,22,18.37500,2,...,1.0,17.000000,17.0,17.0,1.0,17.000000,17.0,17.0,31.0,31.000000,31.0,31.0,NaN,NaN,1.0,28.0,28.0,2018-04-28,2018-04-28,4.0,4.000000,1.0,887.0,887.0,-0.604004,-0.74707,0.799805,-3044996.75,-0.639648,-0.74707,0.799805,-9971130.0,-0.580566,-0.74707,0.799805,-150485.281250,-0.612305,-0.74707,0.799805,-598762.43750
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014168,0.010284,0.142456,3,1123,0,12.031250,15.906250,11.546875,0.000010,0.000013,0.000009,0.03

In [ ]:
test = pd.merge(test, to_join, on='card_id', how='left')
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,quarter_first_active_month,first_active_month_diff_from_today,target,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,transactions_week_of_year_max,...,new_transactions_purchase_hour_nunique,new_transactions_purchase_hour_mean,new_transactions_purchase_hour_min,new_transactions_purchase_hour_max,new_transactions_week_of_year_nunique,new_transactions_week_of_year_mean,new_transactions_week_of_year_min,new_transactions_week_of_year_max,new_transactions_month_diff_sum,new_transactions_month_diff_mean,new_transactions_month_diff_min,new_transactions_month_diff_max,new_transactions_month_diff_var,new_transactions_month_diff_skew,new_transactions_purchase_day_nunique,new_transactions_purchase_day_sum,new_transactions_purchase_day_min,new_transactions_purchase_date_min,new_transactions_purchase_date_max,new_transactions_purchase_month_sum,new_transactions_purchase_month_mean,new_transactions_purchase_month_nunique,new_transactions_purchase_date_max_diff_now,new_transactions_purchase_date_min_diff_now,category_2_mean,category_2_min,category_2_max,category_2_sum,category_3_mean,category_3_min,category_3_max,category_3_sum,new_category_2_mean,new_category_2_min,new_category_2_max,new_category_2_sum,new_category_3_mean,new_category_3_min,new_category_3_max,new_category_3_sum
0,2017-04-01,C_ID_0ab67a22ab,0.010483,0.014168,0.011429,2,1276,NaN,13.375000,18.078125,14.578125,0.000008,0.000011,0.000009,0.036072,0.012024,0.014168,0.010483,0.001915,13.375000,0.000008,18.078125,0.000011,14.578125,0.000009,12,0.176514,0,0.000000,7,190,2.794922,17,14.367188,0,23,24,34.531250,14,52,...,3.0,13.664062,9.0,19.0,3.0,7.000000,5.0,9.0,100.0,33.34375,33.0,34.0,0.333252,1.732422,3.0,47.0,3.0,2018-02-03,2018-02-28,6.0,2.000000,1.0,946.0,971.0,-0.606445,-0.74707,0.799805,-10816712.00,-0.481689,-0.74707,0.799805,-5024630.5,-0.584961,-0.74707,0.799805,-684223.750000,-0.478027,-0.746582,0.799805,-354624.21875
1,2017-01-01,C_ID_130fd0cbdd,0.010612,0.014168,0.010284,1,1366,NaN,14.492188,19.343750,14.046875,0.000008,0.000010,0.000008,0.035065,0.011688,0.014168,0.010284,0.002153,14.492188,0.000008,19.343750,0.000010,14.046875,0.000008,17,0.217896,4,0.051270,7,229,2.935547,18,14.421875,0,23,20,12.437500,2,21,...,8.0,15.218750,9.0,23.0,6.0,12.445312,9.0,16.0,286.0,31.78125,31.0,32.0,0.194458,-1.620117,7.0,102.0,3.0,2018-03-03,2018-04-20,31.0,3.445312,2.0,895.0,943.0,-0.608398,-0.74707,0.799805,-1839159.50,-0.610840,-0.74707,0.799805,-7084146.5,-0.585449,-0.74707,0.799805,-233366.953125,-0.571777,-0.747070,0.799805,-463090.21875
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011383,0.011429,3,1154,NaN,15.171875,13.140625,13.187500,0.000011,0.000010,0.000010,0.035950,0.011986,0.013145,0.011383,0.001004,15.171875,0.000011,13.140625,0.000010,13.187500,0.000010,0,0.000000,0,0.000000,4,34,2.615234,7,17.000000,13,20,7,29.000000,2,49,...,2.0,13.500000,13.0,14.0,2.0,10.000000,9.0,11.0,64.0,32.00000,32.0,32.0,0.000000,NaN,2.0,14.0,1.0,2018-03-01,2018-03-13,6.0,3.000000,1.0,933.0,945.0,-0.604004,-0.74707,0.799805,-3566176.75,-0.325928,-0.74707,0.799805,-3100319.0,-0.583008,-0.74707,0.799805,-417354.500000,-0.407715,-0.746582,0.799805,-273274.71875
3,2017-12-01,C_ID_d27d835a9f,0.010612,0.011383,0.010284,4,1032,NaN,10.953125,11.750000,10.609375,0.000010,0.000011,0.000010,0.032288,0.010

In [ ]:
to_join = reduce_mem_usage(pd.read_csv('New_Data_processed/transactions_other_features_final.csv', index_col=0))
train = pd.merge(train, to_join, on='card_id', how='left')
train.head()

Mem. usage decreased to 32.29 Mb (69.8% reduction)


,first_active_month,card_id,feature_1,feature_2,feature_3,target,quarter_first_active_month,first_active_month_diff_from_today,rare_datapoints,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,...,transactions_category_1_sum,transactions_category_1_mean,transactions_card_id_size,transactions_city_id_nunique,transactions_state_id_nunique,transactions_subsector_id_nunique,transactions_merchant_category_id_nunique,transactions_merchant_id_nunique,transactions_month_lag_sum,transactions_month_lag_mean,transactions_month_lag_min,transactions_month_lag_max,transactions_month_lag_var,transactions_duration_mean,transactions_duration_min,transactions_duration_mean.1,transactions_duration_max,transactions_duration_var,transactions_duration_skew,transactions_amount_month_ratio_mean,transactions_amount_month_ratio_min,transactions_amount_month_ratio_max,transactions_amount_month_ratio_var,transactions_amount_month_ratio_skew,transactions_installments_sum,transactions_installments_mean,transactions_installments_min,transactions_installments_max,transactions_installments_var,transactions_purchase_amount_sum,transactions_purchase_amount_mean,transactions_purchase_amount_min,transactions_purchase_amount_max,transactions_purchase_amount_var,transactions_price_sum,transactions_price_mean,transactions_price_min,transactions_price_max,transactions_price_var,transactions_price_skew
0,2017-06-01,C_ID_92a2005557,0.013145,0.008751,0.011429,-0.820312,2,1215,0,15.968750,10.632812,13.882812,0.000011,0.000007,0.000009,0.033325,0.011108,0.013145,0.008751,0.002214,15.968750,0.000011,10.632812,0.000007,13.882812,0.000009,90,0.346191,8,0.030762,7,835,3.210938,23,13.312500,0,23,35,33.06250,1,...,0.0,0.000000,260,7,3,21,41,94,-1017.0,-3.912109,-8,0,5.750000,-20.265625,-23.656250,-20.265625,25.593750,20.609375,inf,-0.020462,-0.023804,0.024994,0.000021,5.085938,4.0,0.015381,0.0,1.0,0.015205,-167.375000,-0.644043,-0.739258,0.799805,0.020569,-16480.00,-63.375000,-73.93750,80.000000,267.000000,inf
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011383,0.010284,0.392822,1,1366,0,14.632812,15.554688,14.046875,0.000008,0.000008,0.000008,0.032379,0.010796,0.011383,0.010284,0.000556,14.632812,0.000008,15.554688,0.000008,14.046875,0.000008,132,0.377197,17,0.048584,7,1177,3.363281,24,14.718750,0,23,50,25.21875,1,...,31.0,0.088562,350,9,3,24,57,142,-1761.0,-5.031250,-12,0,14.476562,-19.937500,-24.453125,-19.937500,26.406250,61.468750,inf,-0.018997,-0.023193,0.024994,0.000056,3.742188,545.0,1.557617,0.0,10.0,2.259766,-215.375000,-0.615234,-0.742188,0.799805,0.058594,-311.80,-0.891113,-58.93750,0.397949,17.484375,-inf
2,2016-08-01,C_ID_d639edf6cd,0.010612,0.008751,0.010284,0.687988,3,1519,0,16.109375,13.296875,15.617188,0.000007,0.000006,0.000007,0.029648,0.009880,0.010612,0.008751,0.000992,16.109375,0.000007,13.296875,0.000006,15.617188,0.000007,11,0.255859,3,0.069763,7,142,3.302734,14,17.906250,8,23,22,18.37500,2,...,0.0,0.000000,43,5,2,7,8,13,-370.0,-8.601562,-13,0,14.765625,-21.250000,-23.328125,-21.250000,-4.519531,7.816406,5.304688,-0.021652,-0.023560,-0.004704,0.000008,5.730469,0.0,0.000000,0.0,0.0,0.000000,-29.171875,-0.678223,-0.729980,-0.145874,0.007637,-2916.00,-67.812500,-73.00000,-14.585938,76.312500,inf
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014168,0.010284,0.142456,3,1123,0,12.031250,15.906250,11.546875,0.000010,0.000013,0

In [ ]:
test = pd.merge(test, to_join, on='card_id', how='left')
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,quarter_first_active_month,first_active_month_diff_from_today,target,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,transactions_week_of_year_max,...,transactions_category_1_sum,transactions_category_1_mean,transactions_card_id_size,transactions_city_id_nunique,transactions_state_id_nunique,transactions_subsector_id_nunique,transactions_merchant_category_id_nunique,transactions_merchant_id_nunique,transactions_month_lag_sum,transactions_month_lag_mean,transactions_month_lag_min,transactions_month_lag_max,transactions_month_lag_var,transactions_duration_mean,transactions_duration_min,transactions_duration_mean.1,transactions_duration_max,transactions_duration_var,transactions_duration_skew,transactions_amount_month_ratio_mean,transactions_amount_month_ratio_min,transactions_amount_month_ratio_max,transactions_amount_month_ratio_var,transactions_amount_month_ratio_skew,transactions_installments_sum,transactions_installments_mean,transactions_installments_min,transactions_installments_max,transactions_installments_var,transactions_purchase_amount_sum,transactions_purchase_amount_mean,transactions_purchase_amount_min,transactions_purchase_amount_max,transactions_purchase_amount_var,transactions_price_sum,transactions_price_mean,transactions_price_min,transactions_price_max,transactions_price_var,transactions_price_skew
0,2017-04-01,C_ID_0ab67a22ab,0.010483,0.014168,0.011429,2,1276,NaN,13.375000,18.078125,14.578125,0.000008,0.000011,0.000009,0.036072,0.012024,0.014168,0.010483,0.001915,13.375000,0.000008,18.078125,0.000011,14.578125,0.000009,12,0.176514,0,0.000000,7,190,2.794922,17,14.367188,0,23,24,34.531250,14,52,...,23.0,0.338135,68,7,3,12,16,24,-247.0,-3.632812,-8,0,6.027344,-20.031250,-25.296875,-20.031250,8.015625,41.687500,2.328125,-0.017914,-0.022400,0.006931,0.000033,2.310547,141.0,2.074219,1.0,12.0,4.250000,-40.718750,-0.599121,-0.744141,0.235718,0.036957,-34.470,-0.506836,-0.736328,0.117249,0.074280,0.891602
1,2017-01-01,C_ID_130fd0cbdd,0.010612,0.014168,0.010284,1,1366,NaN,14.492188,19.343750,14.046875,0.000008,0.000010,0.000008,0.035065,0.011688,0.014168,0.010284,0.002153,14.492188,0.000008,19.343750,0.000010,14.046875,0.000008,17,0.217896,4,0.051270,7,229,2.935547,18,14.421875,0,23,20,12.437500,2,21,...,2.0,0.025635,78,4,3,12,16,27,-812.0,-10.414062,-13,0,4.687500,-19.828125,-23.421875,-19.828125,9.882812,23.578125,3.748047,-0.020020,-0.023392,0.010284,0.000025,3.779297,83.0,1.064453,1.0,4.0,0.164673,-49.125000,-0.629883,-0.731934,0.318848,0.024017,-48.000,-0.615234,-0.724609,0.315674,0.028961,3.214844
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011383,0.011429,3,1154,NaN,15.171875,13.140625,13.187500,0.000011,0.000010,0.000010,0.035950,0.011986,0.013145,0.011383,0.001004,15.171875,0.000011,13.140625,0.000010,13.187500,0.000010,0,0.000000,0,0.000000,4,34,2.615234,7,17.000000,13,20,7,29.000000,2,49,...,1.0,0.076904,13,4,4,6,8,9,-27.0,-2.076172,-6,0,3.076172,4.316406,-16.625000,4.316406,25.593750,260.500000,0.047333,0.004051,-0.017303,0.024994,0.000256,0.014359,47.0,3.615234,0.0,10.0,11.585938,1.719727,0.132202,-0.536621,0.799805,0.258057,141.800,10.898438,-0.531250,80.000000,583.000000,inf
3,2017-12-01,C_ID_d27d835a9f,0.010612,0.011383,0.010284,4,1032,NaN,10.953125,11.750000,10.609375,0.000010,0.000011,0.000010,0.03228

In [ ]:
to_join = reduce_mem_usage(pd.read_csv('New_Data_processed/new_transactions_other_features_final.csv', index_col=0))
train = pd.merge(train, to_join, on='card_id', how='left')
train.head()

Mem. usage decreased to 30.70 Mb (67.7% reduction)


,first_active_month,card_id,feature_1,feature_2,feature_3,target,quarter_first_active_month,first_active_month_diff_from_today,rare_datapoints,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,...,new_transactions_category_1_sum,new_transactions_category_1_mean,new_transactions_card_id_size,new_transactions_city_id_nunique,new_transactions_state_id_nunique,new_transactions_subsector_id_nunique,new_transactions_merchant_category_id_nunique,new_transactions_merchant_id_nunique,new_transactions_month_lag_sum,new_transactions_month_lag_mean,new_transactions_month_lag_min,new_transactions_month_lag_max,new_transactions_month_lag_var,new_transactions_duration_mean,new_transactions_duration_min,new_transactions_duration_mean.1,new_transactions_duration_max,new_transactions_duration_var,new_transactions_duration_skew,new_transactions_amount_month_ratio_mean,new_transactions_amount_month_ratio_min,new_transactions_amount_month_ratio_max,new_transactions_amount_month_ratio_var,new_transactions_amount_month_ratio_skew,new_transactions_installments_sum,new_transactions_installments_mean,new_transactions_installments_min,new_transactions_installments_max,new_transactions_installments_var,new_transactions_purchase_amount_sum,new_transactions_purchase_amount_mean,new_transactions_purchase_amount_min,new_transactions_purchase_amount_max,new_transactions_purchase_amount_var,new_transactions_price_sum,new_transactions_price_mean,new_transactions_price_min,new_transactions_price_max,new_transactions_price_var,new_transactions_price_skew
0,2017-06-01,C_ID_92a2005557,0.013145,0.008751,0.011429,-0.820312,2,1215,0,15.968750,10.632812,13.882812,0.000011,0.000007,0.000009,0.033325,0.011108,0.013145,0.008751,0.002214,15.968750,0.000011,10.632812,0.000007,13.882812,0.000009,90,0.346191,8,0.030762,7,835,3.210938,23,13.312500,0,23,35,33.06250,1,...,0.0,0.000000,23.0,3.0,1.0,10.0,14.0,23.0,34.0,1.478516,1.0,2.0,0.260986,-18.125000,-23.031250,-18.125000,-9.179688,18.187500,0.885254,-0.018311,-0.023376,-0.009254,1.889467e-05,0.868164,0.0,0.000000,0.0,0.0,0.000000,-13.242188,-0.575684,-0.724609,-0.296143,0.018433,-inf,-inf,-inf,-inf,NaN,NaN
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011383,0.010284,0.392822,1,1366,0,14.632812,15.554688,14.046875,0.000008,0.000008,0.000008,0.032379,0.010796,0.011383,0.010284,0.000556,14.632812,0.000008,15.554688,0.000008,14.046875,0.000008,132,0.377197,17,0.048584,7,1177,3.363281,24,14.718750,0,23,50,25.21875,1,...,0.0,0.000000,6.0,1.0,1.0,4.0,5.0,6.0,9.0,1.500000,1.0,2.0,0.300049,-23.593750,-24.390625,-23.593750,-22.453125,0.625977,0.558594,-0.022339,-0.022934,-0.021927,1.192093e-07,0.000000,6.0,1.000000,1.0,1.0,0.000000,-4.355469,-0.726074,-0.739258,-0.701660,0.000207,-4.355469,-0.725911,-0.739258,-0.701660,0.000207,1.203125
2,2016-08-01,C_ID_d639edf6cd,0.010612,0.008751,0.010284,0.687988,3,1519,0,16.109375,13.296875,15.617188,0.000007,0.000006,0.000007,0.029648,0.009880,0.010612,0.008751,0.000992,16.109375,0.000007,13.296875,0.000006,15.617188,0.000007,11,0.255859,3,0.069763,7,142,3.302734,14,17.906250,8,23,22,18.37500,2,...,0.0,0.000000,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.000000,2.0,2.0,NaN,-21.703125,-21.703125,-21.703125,-21.703125,NaN,NaN,-0.022583,-0.022583,-0.022583,NaN,NaN,0.0,0.000000,0.0,0.0,NaN,-0.700195,-0.700195,-0.700195,-0.700195,NaN,-inf,-inf,-inf,-inf,NaN,NaN
3,2017-09-01,

In [ ]:
test = pd.merge(test, to_join, on='card_id', how='left')
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,quarter_first_active_month,first_active_month_diff_from_today,target,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,transactions_week_of_year_max,...,new_transactions_category_1_sum,new_transactions_category_1_mean,new_transactions_card_id_size,new_transactions_city_id_nunique,new_transactions_state_id_nunique,new_transactions_subsector_id_nunique,new_transactions_merchant_category_id_nunique,new_transactions_merchant_id_nunique,new_transactions_month_lag_sum,new_transactions_month_lag_mean,new_transactions_month_lag_min,new_transactions_month_lag_max,new_transactions_month_lag_var,new_transactions_duration_mean,new_transactions_duration_min,new_transactions_duration_mean.1,new_transactions_duration_max,new_transactions_duration_var,new_transactions_duration_skew,new_transactions_amount_month_ratio_mean,new_transactions_amount_month_ratio_min,new_transactions_amount_month_ratio_max,new_transactions_amount_month_ratio_var,new_transactions_amount_month_ratio_skew,new_transactions_installments_sum,new_transactions_installments_mean,new_transactions_installments_min,new_transactions_installments_max,new_transactions_installments_var,new_transactions_purchase_amount_sum,new_transactions_purchase_amount_mean,new_transactions_purchase_amount_min,new_transactions_purchase_amount_max,new_transactions_purchase_amount_var,new_transactions_price_sum,new_transactions_price_mean,new_transactions_price_min,new_transactions_price_max,new_transactions_price_var,new_transactions_price_skew
0,2017-04-01,C_ID_0ab67a22ab,0.010483,0.014168,0.011429,2,1276,NaN,13.375000,18.078125,14.578125,0.000008,0.000011,0.000009,0.036072,0.012024,0.014168,0.010483,0.001915,13.375000,0.000008,18.078125,0.000011,14.578125,0.000009,12,0.176514,0,0.000000,7,190,2.794922,17,14.367188,0,23,24,34.531250,14,52,...,0.0,0.000000,3.0,3.0,1.0,3.0,3.0,3.0,6.0,2.000000,2.0,2.0,0.000000,-19.671875,-23.828125,-19.671875,-13.031250,33.812500,1.566406,-0.017838,-0.021881,-0.011276,0.000033,1.900391,5.0,1.666992,1.0,3.0,1.333008,-1.777344,-0.592285,-0.722168,-0.383301,0.033447,-1.521810,-0.507270,-0.722168,-0.127767,0.108643,1.686523
1,2017-01-01,C_ID_130fd0cbdd,0.010612,0.014168,0.010284,1,1366,NaN,14.492188,19.343750,14.046875,0.000008,0.000010,0.000008,0.035065,0.011688,0.014168,0.010284,0.002153,14.492188,0.000008,19.343750,0.000010,14.046875,0.000008,17,0.217896,4,0.051270,7,229,2.935547,18,14.421875,0,23,20,12.437500,2,21,...,2.0,0.222168,9.0,2.0,2.0,6.0,8.0,9.0,13.0,1.444336,1.0,2.0,0.277832,-20.937500,-23.703125,-20.937500,-15.695312,5.980469,1.185547,-0.020844,-0.023148,-0.016327,0.000004,2.105469,11.0,1.222656,1.0,3.0,0.444336,-5.945312,-0.660645,-0.740723,-0.506348,0.005062,-5.606771,-0.622975,-0.740723,-0.168783,0.030731,2.683594
2,2017-08-01,C_ID_b709037bc5,0.013145,0.011383,0.011429,3,1154,NaN,15.171875,13.140625,13.187500,0.000011,0.000010,0.000010,0.035950,0.011986,0.013145,0.011383,0.001004,15.171875,0.000011,13.140625,0.000010,13.187500,0.000010,0,0.000000,0,0.000000,4,34,2.615234,7,17.000000,13,20,7,29.000000,2,49,...,1.0,0.500000,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.000000,1.0,1.0,0.000000,1.203125,-23.187500,1.203125,25.593750,1190.000000,NaN,0.001175,-0.022644,0.024994,0.001135,NaN,11.0,5.500000,1.0,10.0,40.500000,0.075195,0.037598,-0.724609,0.799805,1.162109,-0

# Adding some additional features:
<br>
Ref: https://www.kaggle.com/chauhuynh/my-first-kernel-3-699

In [ ]:
train['transactions_purchase_date_max'] = pd.to_datetime(train['transactions_purchase_date_max'])
test['transactions_purchase_date_max'] = pd.to_datetime(test['transactions_purchase_date_max'])
train['new_transactions_purchase_date_max'] = pd.to_datetime(train['new_transactions_purchase_date_max'])
test['new_transactions_purchase_date_max'] = pd.to_datetime(test['new_transactions_purchase_date_max'])

train['transactions_purchase_date_min'] = pd.to_datetime(train['transactions_purchase_date_min'])
test['transactions_purchase_date_min'] = pd.to_datetime(test['transactions_purchase_date_min'])
train['new_transactions_purchase_date_min'] = pd.to_datetime(train['new_transactions_purchase_date_min'])
test['new_transactions_purchase_date_min'] = pd.to_datetime(test['new_transactions_purchase_date_min'])

train['first_active_month'] = pd.to_datetime(train['first_active_month'])
test['first_active_month'] = pd.to_datetime(test['first_active_month'])

In [ ]:
train['purchase_date_diff'] = (train['transactions_purchase_date_max'] - train['transactions_purchase_date_min']).dt.days
test['purchase_date_diff'] = (test['transactions_purchase_date_max'] - test['transactions_purchase_date_min']).dt.days
train['new_purchase_date_diff'] = (train['new_transactions_purchase_date_max'] - train['new_transactions_purchase_date_min']).dt.days
test['new_purchase_date_diff'] = (test['new_transactions_purchase_date_max'] - test['new_transactions_purchase_date_min']).dt.days

train['purchase_date_average'] = (train['purchase_date_diff'])/train['transactions_card_id_size']
test['purchase_date_average'] = (test['purchase_date_diff'])/test['transactions_card_id_size']
train['new_purchase_date_average'] = (train['new_purchase_date_diff'])/train['new_transactions_card_id_size']
test['new_purchase_date_average'] = (test['new_purchase_date_diff'])/test['new_transactions_card_id_size']

train['purchase_date_diff_now'] = (datetime.datetime.today() - train['transactions_purchase_date_max']).dt.days
test['purchase_date_diff_now'] = (datetime.datetime.today() - test['transactions_purchase_date_max']).dt.days
train['new_purchase_date_diff_now'] = (datetime.datetime.today() - train['new_transactions_purchase_date_max']).dt.days
test['new_purchase_date_diff_now'] = (datetime.datetime.today() - test['new_transactions_purchase_date_max']).dt.days

train['purchase_date_diff_now_min'] = (datetime.datetime.today() - train['transactions_purchase_date_min']).dt.days
test['purchase_date_diff_now_min'] = (datetime.datetime.today() - test['transactions_purchase_date_min']).dt.days
train['new_purchase_date_diff_now_min'] = (datetime.datetime.today() - train['new_transactions_purchase_date_min']).dt.days
test['new_purchase_date_diff_now_min'] = (datetime.datetime.today() - test['new_transactions_purchase_date_min']).dt.days

train['first_buy'] = (train['transactions_purchase_date_min'] - train['first_active_month']).dt.days
test['first_buy'] = (test['transactions_purchase_date_min'] - test['first_active_month']).dt.days
train['new_first_buy'] = (train['new_transactions_purchase_date_min'] - train['first_active_month']).dt.days
test['new_first_buy'] = (test['new_transactions_purchase_date_min'] - test['first_active_month']).dt.days

train['last_buy'] = (train['transactions_purchase_date_max'] - train['first_active_month']).dt.days
test['last_buy'] = (test['transactions_purchase_date_max'] - test['first_active_month']).dt.days
train['new_last_buy'] = (train['new_transactions_purchase_date_max'] - train['first_active_month']).dt.days
test['new_last_buy'] = (test['new_transactions_purchase_date_max'] - test['first_active_month']).dt.days

In [ ]:
train['transactions_purchase_date_max'] = train['transactions_purchase_date_max'].astype(np.int64) * 1e-9
train['new_transactions_purchase_date_max'] = train['new_transactions_purchase_date_max'].astype(np.int64) * 1e-9
train['transactions_purchase_date_min'] = train['transactions_purchase_date_min'].astype(np.int64) * 1e-9
train['new_transactions_purchase_date_min'] = train['new_transactions_purchase_date_min'].astype(np.int64) * 1e-9

test['transactions_purchase_date_max'] = test['transactions_purchase_date_max'].astype(np.int64) * 1e-9
test['new_transactions_purchase_date_max'] = test['new_transactions_purchase_date_max'].astype(np.int64) * 1e-9
test['transactions_purchase_date_min'] = test['transactions_purchase_date_min'].astype(np.int64) * 1e-9
test['new_transactions_purchase_date_min'] = test['new_transactions_purchase_date_min'].astype(np.int64) * 1e-9

In [ ]:
train['card_id_total'] = train['new_transactions_card_id_size'] + train['transactions_card_id_size'] 
test['card_id_total'] = test['new_transactions_card_id_size'] + test['transactions_card_id_size'] 
train['card_id_ratio'] = train['new_transactions_card_id_size'] / train['transactions_card_id_size']
test['card_id_ratio'] = test['new_transactions_card_id_size'] / test['transactions_card_id_size']

train['purchase_amount_total'] = train['new_transactions_purchase_amount_sum'] + train['transactions_purchase_amount_sum']
train['purchase_amount_mean'] = train['new_transactions_purchase_amount_mean'] + train['transactions_purchase_amount_mean']
train['purchase_amount_max'] = train['new_transactions_purchase_amount_max'] + train['transactions_purchase_amount_max']
train['purchase_amount_min'] = train['new_transactions_purchase_amount_min'] + train['transactions_purchase_amount_min']
train['purchase_amount_ratio'] = train['new_transactions_purchase_amount_sum'] / train['transactions_purchase_amount_sum']
test['purchase_amount_total'] = test['new_transactions_purchase_amount_sum'] + test['transactions_purchase_amount_sum']
test['purchase_amount_mean'] = test['new_transactions_purchase_amount_mean'] + test['transactions_purchase_amount_mean']
test['purchase_amount_max'] = test['new_transactions_purchase_amount_max'] + test['transactions_purchase_amount_max']
test['purchase_amount_min'] = test['new_transactions_purchase_amount_min'] + test['transactions_purchase_amount_min']
test['purchase_amount_ratio'] = test['new_transactions_purchase_amount_sum'] / test['transactions_purchase_amount_sum']

train['month_diff_mean'] = train['new_transactions_month_diff_mean'] + train['transactions_month_diff_mean']
test['month_diff_mean'] = test['new_transactions_month_diff_mean'] + test['transactions_month_diff_mean']
train['month_diff_ratio'] = train['new_transactions_month_diff_mean'] / train['transactions_month_diff_mean']
test['month_diff_ratio'] = test['new_transactions_month_diff_mean'] / test['transactions_month_diff_mean']
train['month_lag_mean'] = train['new_transactions_month_lag_mean'] + train['transactions_month_lag_mean']
test['month_lag_mean'] = test['new_transactions_month_lag_mean'] + test['transactions_month_lag_mean']
train['month_lag_max'] = train['new_transactions_month_lag_max'] + train['transactions_month_lag_max']
test['month_lag_max'] = test['new_transactions_month_lag_max'] + test['transactions_month_lag_max']
train['month_lag_min'] = train['new_transactions_month_lag_min'] + train['transactions_month_lag_min']
test['month_lag_min'] = test['new_transactions_month_lag_min'] + test['transactions_month_lag_min']

train['category_1_mean'] = train['new_transactions_category_1_mean'] + train['transactions_category_1_mean']
test['category_1_mean'] = test['new_transactions_category_1_mean'] + test['transactions_category_1_mean']
train['category_1_sum'] = train['new_transactions_category_1_sum'] + train['transactions_category_1_sum']
test['category_1_sum'] = test['new_transactions_category_1_sum'] + test['transactions_category_1_sum']

train['installments_mean'] = train['new_transactions_installments_mean'] + train['transactions_installments_mean']
test['installments_mean'] = test['new_transactions_installments_mean'] + test['transactions_installments_mean']
train['installments_total'] = train['new_transactions_installments_sum'] + train['transactions_installments_sum']
test['installments_total'] = test['new_transactions_installments_sum'] + test['transactions_installments_sum']
train['installments_ratio'] = train['new_transactions_installments_sum'] / train['transactions_installments_sum']
test['installments_ratio'] = test['new_transactions_installments_sum'] / test['transactions_installments_sum']
train['installments_max'] = train['new_transactions_installments_max'] + train['transactions_installments_max']
test['installments_max'] = test['new_transactions_installments_max'] + test['transactions_installments_max']
train['installments_min'] = train['new_transactions_installments_min'] + train['transactions_installments_min']
test['installments_min'] = train['new_transactions_installments_min'] + train['transactions_installments_min']

train['duration_mean'] = train['new_transactions_duration_mean'] + train['transactions_duration_mean']
test['duration_mean'] = test['new_transactions_duration_mean'] + test['transactions_duration_mean']
train['duration_max'] = train['new_transactions_duration_max'] + train['transactions_duration_max']
test['duration_max'] = test['new_transactions_duration_max'] + test['transactions_duration_max']
train['duration_min'] = train['new_transactions_duration_min'] + train['transactions_duration_min']
test['duration_min'] = test['new_transactions_duration_min'] + test['transactions_duration_min']

train['amount_month_ratio_mean'] = train['new_transactions_amount_month_ratio_mean'] + train['transactions_amount_month_ratio_mean']
test['amount_month_ratio_mean'] = test['new_transactions_amount_month_ratio_mean'] + test['transactions_amount_month_ratio_mean']
train['amount_month_ratio_min'] = train['new_transactions_amount_month_ratio_min'] + train['transactions_amount_month_ratio_min']
test['amount_month_ratio_min'] = test['new_transactions_amount_month_ratio_min'] + test['transactions_amount_month_ratio_min']
train['amount_month_ratio_max'] = train['new_transactions_amount_month_ratio_max'] + train['transactions_amount_month_ratio_max']
test['amount_month_ratio_max'] = test['new_transactions_amount_month_ratio_max'] + test['transactions_amount_month_ratio_max']

train['CLV'] = train['transactions_card_id_size'] * train['transactions_purchase_amount_sum'] / train['transactions_month_diff_mean']
test['CLV'] = test['transactions_card_id_size'] * test['transactions_purchase_amount_sum'] / test['transactions_month_diff_mean']
train['new_CLV'] = train['new_transactions_card_id_size'] * train['new_transactions_purchase_amount_sum'] / train['new_transactions_month_diff_mean']
test['new_CLV'] = test['new_transactions_card_id_size'] * test['new_transactions_purchase_amount_sum'] / test['new_transactions_month_diff_mean']

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201917 entries, 0 to 201916
Columns: 227 entries, first_active_month to new_CLV
dtypes: datetime64[ns](1), float16(128), float32(5), float64(54), int16(7), int32(3), int64(7), int8(21), object(1)
memory usage: 170.8+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123623 entries, 0 to 123622
Columns: 226 entries, first_active_month to new_CLV
dtypes: datetime64[ns](1), float16(127), float32(5), float64(57), int16(7), int32(3), int64(5), int8(20), object(1)
memory usage: 99.0+ MB


### Merging Merchant Features

#### Merging Numerical Features

In [ ]:
to_join = reduce_mem_usage(pd.read_csv('New_Data_processed/merchant_numerical_features_final.csv', index_col=0))

Mem. usage decreased to 36.63 Mb (49.1% reduction)


In [ ]:
train = pd.merge(train, to_join, on='card_id', how='left')
train.head()

,first_active_month,card_id,target,quarter_first_active_month,first_active_month_diff_from_today,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_0,feature_3_1,feature_1,feature_2,feature_3,year_of_joining_2011,year_of_joining_2012,year_of_joining_2013,year_of_joining_2014,year_of_joining_2015,year_of_joining_2016,year_of_joining_2017,year_of_joining_2018,authorized_flag_sum,authorized_flag_mean,category_1_sum,category_1_mean,category_2_1.0_sum,category_2_1.0_mean,category_2_2.0_sum,category_2_2.0_mean,...,purchase_date_max,purchase_date_max_diff_now,purchase_date_min_diff_now,Unnamed: 0,category_2_mean,category_2_min,category_2_max,category_2_sum,category_3_mean,category_3_min,category_3_max,category_3_sum,merchant_numerical_1_sum,merchant_numerical_1_mean,merchant_numerical_1_min,merchant_numerical_1_max,merchant_numerical_1_std,merchant_numerical_2_sum,merchant_numerical_2_mean,merchant_numerical_2_min,merchant_numerical_2_max,merchant_numerical_2_std,merchant_avg_sales_lag3_sum,merchant_avg_sales_lag3_mean,merchant_avg_sales_lag3_std,merchant_avg_purchases_lag3_sum,merchant_avg_purchases_lag3_mean,merchant_avg_purchases_lag3_std,merchant_avg_sales_lag6_sum,merchant_avg_sales_lag6_mean,merchant_avg_sales_lag6_std,merchant_avg_purchases_lag6_sum,merchant_avg_purchases_lag6_mean,merchant_avg_purchases_lag6_std,merchant_avg_sales_lag12_sum,merchant_avg_sales_lag12_mean,merchant_avg_sales_lag12_std,merchant_avg_purchases_lag12_sum,merchant_avg_purchases_lag12_mean,merchant_avg_purchases_lag12_std
0,2017-06-01,C_ID_92a2005557,-0.820312,2,1213,6065,2426,1213,0.004124,0.001649,0.000824,0,0,0,0,1,0,1,0,0,1,0.013145,0.008751,0.011429,0,0,0,0,0,0,1,0,270.0,0.954102,0.0,0.000000,280.0,0.989258,0.0,0.0,...,2018-04-29,881,1187,13910101.0,-0.278809,-0.74707,164661.515625,-5273924.500,-0.065552,-0.74707,165043.796875,-1062100.750,4688.00,16.281250,-0.057465,183.750000,44.562500,4532.0000,15.734375,-0.057465,182.12500,44.093750,1087.398804,3.775690,17.393085,inf,inf,NaN,2596.271973,9.014833,31.734573,inf,inf,NaN,2811.324463,9.761543,35.244831,inf,inf,NaN
1,2017-01-01,C_ID_3d0044924f,0.392822,1,1364,5456,1364,0,0.002934,0.000733,0.000000,0,0,0,1,0,1,0,0,1,0,0.010712,0.011383,0.010284,0,0,0,0,0,0,1,0,345.0,0.969238,31.0,0.087097,356.0,1.000000,0.0,0.0,...,2018-03-30,911,1359,13220608.0,-0.280273,-0.74707,165298.750000,-5324736.000,-0.311035,-0.74707,123960.789062,-4108867.750,9960.00,26.843750,-0.057465,172.750000,50.250000,9752.0000,26.281250,-0.057465,170.75000,49.781250,452.660004,1.220108,1.065189,518.125666,1.396565,1.901367,761.210022,2.051779,10.825763,1546.266088,4.167833,36.937500,817.449951,2.203369,12.232841,1679.033845,4.525698,40.625000
2,2016-08-01,C_ID_d639edf6cd,0.687988,3,1517,3034,3034,0,0.001318,0.001318,0.000000,0,1,0,0,0,0,1,0,1,0,0.010612,0.008751,0.010284,0,0,0,0,0,1,0,0,42.0,0.954590,0.0,0.000000,4.0,0.090881,0.0,0.0,...,2018-04-28,882,1354,20830802.0,-0.146729,-0.74707,110651.273438,-967283.250,-0.060486,-0.74707,165298.750000,-1002326.625,53.75,1.194336,-0.057465,21.828125,4.484375,46.0625,1.023438,-0.057465,21.59375,4.484375,51.360001,1.141333,0.884468,57.970595,1.288235,1.741211,53.169998,1.181556,1.105324,61.972721,1.377172,2.208984,53.689999,1.193111,1.127176,62.831363,1.396253,2.261719
3,2017-09-01,C_ID_186d6a6901,0.142456,3,1121,4484,3363,0,0.003569,0.002676,0.000000,0,0,0,1,0,0,0,1,1,0,0.010712,0.014168,0.010284,0,0,0,0,0,0,1,0,84.0,1.000000,13.0,0.154785,25.0,0.297607,0.0,0.0,...,2018-04-18,892,1096,18387612.0,-0.213013,-0.74707,133425.281250,-1946974.625,-0.362061,-0.74707,138262.671875,-4627596.500,368.00,3.833984,-0.057465,27.578125,8.593750,360.2500,3.753906,-0.057465,27.31250,8.507812,150.750000,1.570312,1.739451,204.211116,2.127199,3.421875,162.660004,1.694375,2.181952,230.818389,2.404358,4.351562,163.869995,1.706979,2.230313,234.084487,2.438380,

In [ ]:
test = pd.merge(test, to_join, on='card_id', how='left')
test.head()

,first_active_month,card_id,quarter_first_active_month,first_active_month_diff_from_today,target,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_0,feature_3_1,feature_1,feature_2,feature_3,year_of_joining_2011,year_of_joining_2012,year_of_joining_2013,year_of_joining_2014,year_of_joining_2015,year_of_joining_2016,year_of_joining_2017,year_of_joining_2018,authorized_flag_sum,authorized_flag_mean,category_1_sum,category_1_mean,category_2_1.0_sum,category_2_1.0_mean,category_2_2.0_sum,category_2_2.0_mean,...,purchase_date_max,purchase_date_max_diff_now,purchase_date_min_diff_now,Unnamed: 0,category_2_mean,category_2_min,category_2_max,category_2_sum,category_3_mean,category_3_min,category_3_max,category_3_sum,merchant_numerical_1_sum,merchant_numerical_1_mean,merchant_numerical_1_min,merchant_numerical_1_max,merchant_numerical_1_std,merchant_numerical_2_sum,merchant_numerical_2_mean,merchant_numerical_2_min,merchant_numerical_2_max,merchant_numerical_2_std,merchant_avg_sales_lag3_sum,merchant_avg_sales_lag3_mean,merchant_avg_sales_lag3_std,merchant_avg_purchases_lag3_sum,merchant_avg_purchases_lag3_mean,merchant_avg_purchases_lag3_std,merchant_avg_sales_lag6_sum,merchant_avg_sales_lag6_mean,merchant_avg_sales_lag6_std,merchant_avg_purchases_lag6_sum,merchant_avg_purchases_lag6_mean,merchant_avg_purchases_lag6_std,merchant_avg_sales_lag12_sum,merchant_avg_sales_lag12_mean,merchant_avg_sales_lag12_std,merchant_avg_purchases_lag12_sum,merchant_avg_purchases_lag12_mean,merchant_avg_purchases_lag12_std
0,2017-04-01,C_ID_0ab67a22ab,2,1274,NaN,3822,3822,1274,0.002356,0.002356,0.000785,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,47.0,0.662109,23.0,0.323975,71.0,1.000000,0.0,0.0,...,2018-02-28,941,1271,2560453.25,-0.280273,-0.74707,165298.750000,-5.324736e+06,-0.249878,-0.74707,109778.484375,-3.471503e+06,5.582031,0.075439,-0.057465,2.619141,0.351318,5.148438,0.069580,-0.057465,2.589844,0.350098,104.019997,1.405676,1.731793,113.176425,1.529411,2.361328,114.110001,1.542027,2.411579,123.725782,1.671970,3.017578,109.040001,1.473513,2.394857,119.372364,1.613140,3.099609
1,2017-01-01,C_ID_130fd0cbdd,1,1364,NaN,2728,4092,0,0.001467,0.002199,0.000000,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,86.0,0.988281,4.0,0.045990,4.0,0.045990,0.0,0.0,...,2018-04-20,890,1352,25592514.00,-0.185181,-0.74707,122320.234375,-7.246192e+05,-0.400146,-0.74707,143270.734375,-5.044016e+06,156.000000,1.773438,-0.057465,12.914062,3.931641,154.375000,1.754883,-0.057465,12.890625,3.919922,95.540001,1.085682,0.637843,102.186709,1.161213,1.247070,96.959999,1.101818,0.798999,106.378598,1.208848,1.582031,97.239998,1.105000,0.813341,109.227951,1.241227,1.655273
2,2017-08-01,C_ID_b709037bc5,3,1152,NaN,5760,1152,1152,0.004341,0.000868,0.000868,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,11.0,0.733398,2.0,0.133301,3.0,0.199951,0.0,0.0,...,2018-03-13,928,1128,27381482.00,-0.162842,-0.74707,117208.968750,-1.490178e+06,-0.000525,-0.74707,73731.148438,-7.638367e+05,1.675781,0.111755,-0.057465,1.697266,0.445557,1.021484,0.068115,-0.057465,1.637695,0.434570,15.610000,1.040667,0.137346,15.816781,1.054452,0.138184,14.910000,0.994000,0.201487,16.508767,1.100584,0.228638,14.820000,0.988000,0.256910,16.574818,1.104988,0.266846
3,2017-12-01,C_ID_d27d835a9f,4,1030,NaN,2060,1030,0,0.001942,0.000971,0.000000,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,36.0,1.000000,1.0,0.027771,35.0,0.972168,0.0,0.0,...,2018-04-17,893,1027,27575152.00,-0.276123,-0.74707,163628.968750,-5.191592e+06,-0.225098,-0.74707,109694.171875,-3.185548e+06,222.375000,6.175781,-0.057465,172.750000,28.812500,210.125000,5.835938,-0.057465,170.750000,28.515625,35.660000,0.990556,0.068450,36.912014,1.025334,0.058624,35.130001,0.975833,0.097845,36.856241,1.023784,0.092590,36.970001,1.026944,0.138931,38.487770,1.069105,0.136475
4,2015-12-01,C_ID_2b5e3df5c2,4,1761,NaN,8805,1761,17

#### Merging Categorical Features

In [ ]:
to_join = reduce_mem_usage(pd.read_csv('New_Data_processed/merchant_categorical_features_final.csv', index_col=0))

Mem. usage decreased to 69.85 Mb (75.8% reduction)


In [ ]:
train = pd.merge(train, to_join, on='card_id', how='left')
train.head()

,first_active_month,card_id,target,quarter_first_active_month,first_active_month_diff_from_today,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_0,feature_3_1,feature_1,feature_2,feature_3,year_of_joining_2011,year_of_joining_2012,year_of_joining_2013,year_of_joining_2014,year_of_joining_2015,year_of_joining_2016,year_of_joining_2017,year_of_joining_2018,authorized_flag_sum,authorized_flag_mean,category_1_sum,category_1_mean,category_2_1.0_sum,category_2_1.0_mean,category_2_2.0_sum,category_2_2.0_mean,...,merchant_active_months_lag6_5.0_std,merchant_active_months_lag6_6.0_sum,merchant_active_months_lag6_6.0_mean,merchant_active_months_lag6_6.0_std,merchant_active_months_lag12_1.0_sum,merchant_active_months_lag12_1.0_mean,merchant_active_months_lag12_1.0_std,merchant_active_months_lag12_2.0_sum,merchant_active_months_lag12_2.0_mean,merchant_active_months_lag12_2.0_std,merchant_active_months_lag12_3.0_sum,merchant_active_months_lag12_3.0_mean,merchant_active_months_lag12_3.0_std,merchant_active_months_lag12_4.0_sum,merchant_active_months_lag12_4.0_mean,merchant_active_months_lag12_4.0_std,merchant_active_months_lag12_5.0_sum,merchant_active_months_lag12_5.0_mean,merchant_active_months_lag12_5.0_std,merchant_active_months_lag12_6.0_sum,merchant_active_months_lag12_6.0_mean,merchant_active_months_lag12_6.0_std,merchant_active_months_lag12_7.0_sum,merchant_active_months_lag12_7.0_mean,merchant_active_months_lag12_7.0_std,merchant_active_months_lag12_8.0_sum,merchant_active_months_lag12_8.0_mean,merchant_active_months_lag12_8.0_std,merchant_active_months_lag12_9.0_sum,merchant_active_months_lag12_9.0_mean,merchant_active_months_lag12_9.0_std,merchant_active_months_lag12_10.0_sum,merchant_active_months_lag12_10.0_mean,merchant_active_months_lag12_10.0_std,merchant_active_months_lag12_11.0_sum,merchant_active_months_lag12_11.0_mean,merchant_active_months_lag12_11.0_std,merchant_active_months_lag12_12.0_sum,merchant_active_months_lag12_12.0_mean,merchant_active_months_lag12_12.0_std
0,2017-06-01,C_ID_92a2005557,-0.820312,2,1213,6065,2426,1213,0.004124,0.001649,0.000824,0,0,0,0,1,0,1,0,0,1,0.013145,0.008751,0.011429,0,0,0,0,0,0,1,0,270.0,0.954102,0.0,0.000000,280.0,0.989258,0.0,0.0,...,0.058929,287.0,0.996582,0.058929,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,1,0.003471,0.058929,0,0.0,0.0,5.0,0.017365,0.130859,0,0.0,0.0,0,0.000000,0.000000,2,0.006943,0.083191,0,0.0,0.0,280.0,0.972168,0.164673
1,2017-01-01,C_ID_3d0044924f,0.392822,1,1364,5456,1364,0,0.002934,0.000733,0.000000,0,0,0,1,0,1,0,0,1,0,0.010712,0.011383,0.010284,0,0,0,0,0,0,1,0,345.0,0.969238,31.0,0.087097,356.0,1.000000,0.0,0.0,...,0.000000,371.0,1.000000,0.000000,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.000000,0.000000,0,0.0,0.0,9.0,0.024261,0.154053,0,0.0,0.0,0,0.000000,0.000000,3,0.008087,0.089661,0,0.0,0.0,359.0,0.967773,0.177124
2,2016-08-01,C_ID_d639edf6cd,0.687988,3,1517,3034,3034,0,0.001318,0.001318,0.000000,0,1,0,0,0,0,1,0,1,0,0.010612,0.008751,0.010284,0,0,0,0,0,1,0,0,42.0,0.954590,0.0,0.000000,4.0,0.090881,0.0,0.0,...,0.000000,45.0,1.000000,0.000000,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.000000,0.000000,0,0.0,0.0,1.0,0.022217,0.149048,0,0.0,0.0,0,0.000000,0.000000,1,0.022217,0.149048,0,0.0,0.0,43.0,0.955566,0.208374
3,2017-09-01,C_ID_186d6a6901,0.142456,3,1121,4484,3363,0,0.003569,0.002676,0.000000,0,0,0,1,0,0,0,1,1,0,0.010712,0.014168,0.010284,0,0,0,0,0,0,1,0,84.0,1.000000,13.0,0.154785,25.0,0.297607,0.0,0.0,...,0.000000,96.0,1.000000,0.000000,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.000000,0.000000,0,0.0,0.0,9.0,0.093750,0.292969,0,0.0,0.0,3,0.031250,0.174927,2,0.020828,0.143555,0,0.0,0.0,82.0,0.854004,0.354736
4,2017-11-01,C_ID_cdbd2c0db2,-0.159790,4,1060,1060,3180,0,0.000943,0.002831,0.000000,1,0,0,0,0,0,0,1,1,0,0.008057,0.014168,0.010284,0,0,0,0,0,0,1,0,164.0,0.970215,17.0,0.100586,29.0,0.171631

In [ ]:
test = pd.merge(test, to_join, on='card_id', how='left')
test.head()

,first_active_month,card_id,quarter_first_active_month,first_active_month_diff_from_today,target,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_1_1,feature_1_2,feature_1_3,feature_1_4,feature_1_5,feature_2_1,feature_2_2,feature_2_3,feature_3_0,feature_3_1,feature_1,feature_2,feature_3,year_of_joining_2011,year_of_joining_2012,year_of_joining_2013,year_of_joining_2014,year_of_joining_2015,year_of_joining_2016,year_of_joining_2017,year_of_joining_2018,authorized_flag_sum,authorized_flag_mean,category_1_sum,category_1_mean,category_2_1.0_sum,category_2_1.0_mean,category_2_2.0_sum,category_2_2.0_mean,...,merchant_active_months_lag6_5.0_std,merchant_active_months_lag6_6.0_sum,merchant_active_months_lag6_6.0_mean,merchant_active_months_lag6_6.0_std,merchant_active_months_lag12_1.0_sum,merchant_active_months_lag12_1.0_mean,merchant_active_months_lag12_1.0_std,merchant_active_months_lag12_2.0_sum,merchant_active_months_lag12_2.0_mean,merchant_active_months_lag12_2.0_std,merchant_active_months_lag12_3.0_sum,merchant_active_months_lag12_3.0_mean,merchant_active_months_lag12_3.0_std,merchant_active_months_lag12_4.0_sum,merchant_active_months_lag12_4.0_mean,merchant_active_months_lag12_4.0_std,merchant_active_months_lag12_5.0_sum,merchant_active_months_lag12_5.0_mean,merchant_active_months_lag12_5.0_std,merchant_active_months_lag12_6.0_sum,merchant_active_months_lag12_6.0_mean,merchant_active_months_lag12_6.0_std,merchant_active_months_lag12_7.0_sum,merchant_active_months_lag12_7.0_mean,merchant_active_months_lag12_7.0_std,merchant_active_months_lag12_8.0_sum,merchant_active_months_lag12_8.0_mean,merchant_active_months_lag12_8.0_std,merchant_active_months_lag12_9.0_sum,merchant_active_months_lag12_9.0_mean,merchant_active_months_lag12_9.0_std,merchant_active_months_lag12_10.0_sum,merchant_active_months_lag12_10.0_mean,merchant_active_months_lag12_10.0_std,merchant_active_months_lag12_11.0_sum,merchant_active_months_lag12_11.0_mean,merchant_active_months_lag12_11.0_std,merchant_active_months_lag12_12.0_sum,merchant_active_months_lag12_12.0_mean,merchant_active_months_lag12_12.0_std
0,2017-04-01,C_ID_0ab67a22ab,2,1274,NaN,3822,3822,1274,0.002356,0.002356,0.000785,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,47.0,0.662109,23.0,0.323975,71.0,1.000000,0.0,0.0,...,0.0,74.0,1.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,3.0,0.040527,0.198608,0,0.00000,0.000000,0,0.0,0.0,0,0.0000,0.000000,0,0.0000,0.000000,71.0,0.959473,0.198608
1,2017-01-01,C_ID_130fd0cbdd,1,1364,NaN,2728,4092,0,0.001467,0.002199,0.000000,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,86.0,0.988281,4.0,0.045990,4.0,0.045990,0.0,0.0,...,0.0,88.0,1.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,1.0,0.011360,0.106628,1,0.01136,0.106628,0,0.0,0.0,0,0.0000,0.000000,0,0.0000,0.000000,86.0,0.977051,0.149902
2,2017-08-01,C_ID_b709037bc5,3,1152,NaN,5760,1152,1152,0.004341,0.000868,0.000868,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,11.0,0.733398,2.0,0.133301,3.0,0.199951,0.0,0.0,...,0.0,15.0,1.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.000000,0,0.00000,0.000000,0,0.0,0.0,0,0.0000,0.000000,0,0.0000,0.000000,15.0,1.000000,0.000000
3,2017-12-01,C_ID_d27d835a9f,4,1030,NaN,2060,1030,0,0.001942,0.000971,0.000000,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,36.0,1.000000,1.0,0.027771,35.0,0.972168,0.0,0.0,...,0.0,36.0,1.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.000000,0.000000,0,0.00000,0.000000,0,0.0,0.0,0,0.0000,0.000000,0,0.0000,0.000000,36.0,1.000000,0.000000
4,2015-12-01,C_ID_2b5e3df5c2,4,1761,NaN,8805,1761,1761,0.002840,0.000568,0.000568,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,93.0,0.801758,0.0,0.000000,4.0,0.034485,0.0,0.0,...,0.0,119.0,1.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,7.0,0.058838,0.236328,1,0.00840,0.091675,0,0.0,0.0,1,0.0084,0.091675,1,0.0084,0.091675,109.0,0.916016,0.278564


Check for NA and imputating the NA values if needed

In [ ]:
na_columns = train.columns[train.isna().any()]

In [ ]:
for column in na_columns:
  train[column].fillna(train[column].mode()[0], inplace=True)
  test[column].fillna(test[column].mode()[0], inplace=True)

In [ ]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,quarter_first_active_month,first_active_month_diff_from_today,rare_datapoints,cat_time_feature1,cat_time_feature2,cat_time_feature3,cat_time_ratio1,cat_time_ratio2,cat_time_ratio3,feature_sum,feature_mean,feature_max,feature_min,feature_std,days_feature_1,days_feature_1_ratio,days_feature_2,days_feature_2_ratio,days_feature_3,days_feature_3_ratio,transactions_is_weekend_sum,transactions_is_weekend_mean,transactions_purchase_on_holiday_sum,transactions_purchase_on_holiday_mean,transactions_weekday_nunique,transactions_weekday_sum,transactions_weekday_mean,transactions_purchase_hour_nunique,transactions_purchase_hour_mean,transactions_purchase_hour_min,transactions_purchase_hour_max,transactions_week_of_year_nunique,transactions_week_of_year_mean,transactions_week_of_year_min,...,new_transactions_price_skew,purchase_date_diff,new_purchase_date_diff,purchase_date_average,new_purchase_date_average,purchase_date_diff_now,new_purchase_date_diff_now,purchase_date_diff_now_min,new_purchase_date_diff_now_min,first_buy,new_first_buy,last_buy,new_last_buy,card_id_total,card_id_ratio,purchase_amount_total,purchase_amount_mean,purchase_amount_max,purchase_amount_min,purchase_amount_ratio,month_diff_mean,month_diff_ratio,month_lag_mean,month_lag_max,month_lag_min,category_1_mean,category_1_sum,installments_mean,installments_total,installments_ratio,installments_max,installments_min,duration_mean,duration_max,duration_min,amount_month_ratio_mean,amount_month_ratio_min,amount_month_ratio_max,CLV,new_CLV
0,2017-06-01,C_ID_92a2005557,0.013145,0.008751,0.011429,-0.820312,2,1215,0,15.968750,10.632812,13.882812,0.000011,0.000007,0.000009,0.033325,0.011108,0.013145,0.008751,0.002214,15.968750,0.000011,10.632812,0.000007,13.882812,0.000009,90,0.346191,8,0.030762,7,835,3.210938,23,13.312500,0,23,35,33.06250,1,...,1.732422,243,55.0,0.934615,2.391304,949,886,1192,941.0,26,277.0,269,332,283.0,0.088462,-180.62500,-1.219727,0.503906,-1.463867,0.079102,63.00000,1.000977,-2.433594,2.0,-7.0,0.000000,0.0,0.015381,4.0,0.000000,1.0,0.0,-38.37500,16.406250,-46.68750,-0.038757,-0.047180,0.015747,-1382.193604,-9.664105
1,2017-01-01,C_ID_3d0044924f,0.010712,0.011383,0.010284,0.392822,1,1366,0,14.632812,15.554688,14.046875,0.000008,0.000008,0.000008,0.032379,0.010796,0.011383,0.010284,0.000556,14.632812,0.000008,15.554688,0.000008,14.046875,0.000008,132,0.377197,17,0.048584,7,1177,3.363281,24,14.718750,0,23,50,25.21875,1,...,1.203125,390,57.0,1.114286,9.500000,974,916,1364,973.0,5,396.0,395,453,356.0,0.017143,-219.75000,-1.341797,0.098145,-1.481445,0.020218,64.87500,1.002930,-3.531250,2.0,-11.0,0.088562,31.0,2.558594,551.0,0.011009,11.0,1.0,-43.53125,3.953125,-48.84375,-0.041321,-0.046143,0.003067,-2326.133057,-0.804087
2,2016-08-01,C_ID_d639edf6cd,0.010612,0.008751,0.010284,0.687988,3,1519,0,16.109375,13.296875,15.617188,0.000007,0.000006,0.000007,0.029648,0.009880,0.010612,0.008751,0.000992,16.109375,0.000007,13.296875,0.000006,15.617188,0.000007,11,0.255859,3,0.069763,7,142,3.302734,14,17.906250,8,23,22,18.37500,2,...,1.732422,412,0.0,9.581395,0.000000,947,887,1359,887.0,163,635.0,575,635,44.0,0.023256,-29.87500,-1.378906,-0.846191,-1.429688,0.024002,62.31250,0.989746,-6.601562,2.0,-11.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,-42.93750,-26.218750,-45.03125,-0.044250,-0.046143,-0.027283,-40.040398,-0.022587
3,2017-09-01,C_ID_186d6a6901,0.010712,0.014168,0.010284,0.142456,3,1123,0,12.031250,15.906250,11.546875,0.000010,0.000013,0.000009,0.035156,0.011719,0.014168,0.010284,0.002129,12.031250,0.000010,15.906250,0.000013,11.546875,0.000009,11,0.142822,0,0.000000,7,215,2.792969,16,14.445312,0,23,20,32.00000,1,...,1.732422,155,42.0,2.012987,6.000000,946,897,1101,939.0,25,187.0,180,229,84.0,0.090909,-54.78125,-1.316406,0.232910,-1.474609,0.092896,62.96875,1.005859,-1.118164,2.0,-4.0,0.298828,13.0,1.974609,92.0,0.069763,4.0,0.0,-41.37500,7.218750,-47.12500,-0.041901,-0.046997,0.007523,-122.954704,-1.0321

## Check for Nan and impute if needed

In [ ]:
train.columns[train.isna().any()]

Index([], dtype='object')

In [ ]:
test.columns[test.isna().any()]

Index(['target', 'new_purchase_date_diff_now', 'new_last_buy'], dtype='object')

In [ ]:
train.columns[train.isna().any()]

Index([], dtype='object')

In [ ]:
test.columns[test.isna().any()]

Index(['target'], dtype='object')

In [ ]:
train.drop(columns=['first_active_month'], inplace=True)
test.drop(columns=['first_active_month'], inplace=True)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201917 entries, 0 to 201916
Columns: 226 entries, card_id to new_CLV
dtypes: float16(128), float32(5), float64(54), int16(7), int32(3), int64(7), int8(21), object(1)
memory usage: 169.2+ MB


Saving two sets of files. One with the merchant features and one without it. 

In [ ]:
train.to_csv('New_Data_processed/train_features_without_merchant.csv')
test.to_csv('New_Data_processed/test_features_without_merchant.csv')

In [ ]:
train.to_csv('New_Data_processed/train_features_generated.csv')
test.to_csv('New_Data_processed/test_features_generated.csv')